# 🚀 Azure APIM + MCP 서버 워크샵

## 엔터프라이즈급 Model Context Protocol 서버 구축 실습

![Workshop Banner](images/architecture.png)

---

## 📋 워크샵 개요

이 워크샵에서는 **Azure API Management (APIM)**와 **Azure Functions**를 활용하여 엔터프라이즈급 **MCP (Model Context Protocol)** 서버를 구축합니다.

### 학습 목표

1. **MCP 프로토콜 이해** - Model Context Protocol의 기본 개념과 구조
2. **Azure Functions MCP 서버** - HTTP Trigger 기반 MCP 서버 구현
3. **APIM 게이트웨이 구성** - 인증, Rate Limiting, 변환 정책 적용
4. **REST API → MCP 변환** - 기존 REST API를 MCP 서버로 노출
5. **엔터프라이즈 보안** - OpenID Connect, JWT 검증, External Entra ID
6. **모니터링 & 운영** - Application Insights, APIM Diagnostics

---

## 🎯 워크샵 구조

| 섹션 | 주제 | 소요시간 |
|------|------|----------|
| **Section 0** | 환경 설정 및 사전 준비 | 10분 |
| **Section 1** | Azure Functions로 MCP 서버 개발 | 30분 |
| **Section 2** | APIM으로 엔터프라이즈 게이트웨이 구성 | 20분 |
| **Section 3** | 외부 REST API를 MCP 서버로 변환 | 25분 |
| **Section 4** | OpenID Connect 기반 인증 | 30분 |
| **Section 5** | External Entra ID 연계 (B2B) | 25분 |
| **Section 6** | 모니터링 및 최적화 | 15분 |
| **Section 7** | 리소스 정리 | 5분 |

---

## 🔧 사전 요구사항

### 필수 권한
- ✅ Azure Subscription Contributor 권한
- ✅ Entra ID Application 등록 권한
- ✅ APIM, Functions, Managed Identity 생성 권한

### 필수 도구
- ✅ [Azure CLI](https://docs.microsoft.com/cli/azure/) (최신 버전)
- ✅ [Python 3.12+](https://www.python.org/)
- ✅ [VS Code](https://code.visualstudio.com/) + [Azure Functions Extension](https://marketplace.visualstudio.com/items?itemName=ms-azuretools.vscode-azurefunctions)
- ✅ [Postman](https://www.postman.com/) 또는 REST Client (선택)

### 생성할 Azure 리소스
- Azure API Management (Basicv2 SKU)
- Azure Functions App (Python 3.12)
- Storage Account
- Application Insights
- Entra ID Applications (API + Client)

---

## 🏗️ 아키텍처

```
┌─────────────────┐
│   MCP Client    │
│  (GitHub        │
│   Copilot /     │
│   AI Agent)     │
└────────┬────────┘
         │ HTTPS + Bearer Token
         │
         ▼
┌────────────────────────────────────────────┐
│      Azure API Management (Gateway)        │
│  ┌──────────────────────────────────────┐  │
│  │ 🔐 JWT Validation (OpenID Connect)  │  │
│  │ 🚦 Rate Limiting & Quota            │  │
│  │ 🔄 Request/Response Transform       │  │
│  │ 📊 Logging & Diagnostics            │  │
│  │ 🌐 CORS & Header Management         │  │
│  └──────────────────────────────────────┘  │
└────────┬───────────────────────────────────┘
         │
    ┌────┴────┐
    ▼         ▼
┌──────────┐  ┌─────────────────┐
│  Azure   │  │  External       │
│ Functions│  │  REST APIs      │
│   MCP    │  │  (Weather,      │
│  Server  │  │   GitHub,       │
│          │  │   Custom APIs)  │
└──────────┘  └─────────────────┘
    │
    ▼
┌──────────────────┐
│ Application      │
│ Insights         │
│ (Monitoring)     │
└──────────────────┘
```

---

## 📚 참고 자료

- [Model Context Protocol Specification](https://spec.modelcontextprotocol.io/)
- [Azure API Management 문서](https://learn.microsoft.com/azure/api-management/)
- [Azure Functions 문서](https://learn.microsoft.com/azure/azure-functions/)
- [Microsoft Entra ID 문서](https://learn.microsoft.com/entra/identity/)

---

**준비되셨나요? 시작하겠습니다! 🎉**

<a id='0'></a>
# 0️⃣ 환경 설정 및 초기화

이 섹션에서는 워크샵에 필요한 환경을 설정하고 Azure 리소스를 생성합니다.

In [ ]:
# 필수 라이브러리 임포트
import os
import json
import subprocess
from datetime import datetime
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 환경 설정
environment = os.getenv("ENVIRONMENT", "dev")

# 워크샵 설정
WORKSHOP_CONFIG = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID", ""),
    "tenant_id": os.getenv("AZURE_TENANT_ID", ""),
    "location": os.getenv("AZURE_LOCATION", "koreacentral"),
    "resource_group": os.getenv("RESOURCE_GROUP_NAME", "rg-mcp-lab"),
    "environment": environment,
    "mcp_protocol_version": "2024-11-05"
}

print("✅ 워크샵 설정 로드 완료")
# print(json.dumps(WORKSHOP_CONFIG, indent=2, ensure_ascii=False))

: 

In [ ]:
# Azure CLI 로그인 확인
result = subprocess.run(["az", "account", "show"], capture_output=True, text=True)

if result.returncode != 0:
    print("⚠️ Azure CLI 로그인이 필요합니다. 다음 명령어를 실행하세요:")
    print("   az login")
else:
    account = json.loads(result.stdout)
    print(f"✅ 로그인 완료: {account['user']['name'][0:10]}")
    print(f"   구독: {account['name'][0:10]}")
    print(f"   구독 ID: {account['id'][0:10]}")
    
    # 설정 파일에 구독 ID 저장
    WORKSHOP_CONFIG["subscription_id"] = account['id']
    WORKSHOP_CONFIG["tenant_id"] = account['tenantId']

: 

: 

## 0.1 Azure CLI 로그인 및 구독 설정

In [3]:
# 리소스 그룹 생성
rg_name = WORKSHOP_CONFIG["resource_group"]
location = WORKSHOP_CONFIG["location"]

print(f"📦 리소스 그룹 생성: {rg_name}")
result = subprocess.run([
    "az", "group", "create",
    "--name", rg_name,
    "--location", location
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ 리소스 그룹 생성 완료: {rg_name}")
else:
    print(f"ℹ️ 리소스 그룹이 이미 존재하거나 생성 중 오류 발생")
    print(result.stderr)

📦 리소스 그룹 생성: rg-mcp-lab
✅ 리소스 그룹 생성 완료: rg-mcp-lab


<a id='1'></a>
# 1️⃣ Azure Functions로 MCP 서버 개발

이 섹션에서는 Azure Functions를 사용하여 MCP (Model Context Protocol) 서버를 구현하고 배포합니다.

## 학습 목표
- MCP 프로토콜의 기본 구조 이해
- HTTP Trigger Function으로 MCP 엔드포인트 구현
- 로컬 테스트 및 Azure 배포

---
<a id='1.1'></a>
## 1.1 MCP 서버 코드 작성

**Azure Functions**를 사용하여 MCP (Model Context Protocol) 서버를 구현합니다.

### 왜 Azure Functions인가?

| 특징 | 설명 |
|------|------|
| **서버리스** | 인프라 관리 없이 코드에만 집중 |
| **자동 스케일링** | 요청량에 따라 자동으로 인스턴스 확장/축소 |
| **종량제 과금** | 실행 시간과 메모리 사용량 기준 과금 (Consumption Plan) |
| **Managed Identity** | Azure 리소스에 안전하게 접근 (키 노출 없음) |
| **통합 모니터링** | Application Insights로 로그, 메트릭, 추적 자동 수집 |

### HTTP Trigger 특징

MCP 서버는 **HTTP Trigger**를 사용하여 REST API를 제공합니다:

```python
@app.route(route="mcp/tools", methods=["GET", "POST"])
def tools_endpoint(req: func.HttpRequest) -> func.HttpResponse:
    ...
```

| 설정 | 값 | 설명 |
|------|-----|------|
| **Auth Level** | `FUNCTION` | Function Key로 인증 (URL에 `?code=xxx` 필요) |
| **Route** | `mcp/*` | `/api/mcp/health`, `/api/mcp/tools` 등 |
| **Methods** | `GET`, `POST` | RESTful API 패턴 지원 |

### 구현된 MCP 엔드포인트

| 엔드포인트 | 메서드 | 설명 |
|-----------|--------|------|
| `/api/mcp/health` | GET | 서버 상태 확인 (Health Check) |
| `/api/mcp/info` | GET | 서버 정보 및 지원 기능 반환 |
| `/api/mcp/tools` | GET | 사용 가능한 도구 목록 반환 |
| `/api/mcp/tools` | POST | 도구 실행 (name, arguments) |
| `/api/mcp/messages` | POST | JSON-RPC 스타일 메시지 처리 |

### 구현된 MCP Tools

| 도구 | 설명 | 필수 파라미터 |
|------|------|---------------|
| `echo` | 입력 메시지 그대로 반환 (테스트용) | `message` |
| `get_current_time` | 현재 시간 반환 (UTC/KST) | - |
| `calculate` | 사칙연산 수행 | `a`, `b`, `operation` |

### MCP 프로토콜 버전

```python
MCP_PROTOCOL_VERSION = "2024-11-05"
```

이 워크샵은 **MCP 2024-11-05** 스펙을 준수합니다.

> 💡 다음 셀에서 MCP 서버 코드를 `src/mcp-function/function_app.py`에 저장합니다.

In [4]:
%%writefile src/mcp-function/function_app.py
"""
Azure Functions 기반 MCP (Model Context Protocol) 서버

이 모듈은 Azure Functions를 사용하여 MCP 프로토콜을 구현한 서버입니다.
MCP는 AI 에이전트가 외부 도구와 상호작용할 수 있도록 하는 표준 프로토콜입니다.

주요 기능:
- MCP 2024-11-05 프로토콜 준수
- HTTP 기반 REST API 제공
- 3가지 기본 도구 (echo, get_current_time, calculate)
- JSON-RPC 스타일 메시지 처리
"""

import azure.functions as func
import json
import logging
from datetime import datetime
from typing import Dict, List, Any

# ============================================================
# 상수 및 설정
# ============================================================

# MCP 프로토콜 버전 (2024년 11월 5일 스펙)
MCP_PROTOCOL_VERSION = "2024-11-05"

# Azure Functions 앱 초기화
# - http_auth_level: FUNCTION 레벨 인증 요구 (URL에 코드 필요)
app = func.FunctionApp(http_auth_level=func.AuthLevel.FUNCTION)

# ============================================================
# MCP 서버 메타데이터
# ============================================================

MCP_SERVER_INFO = {
    "name": "Azure-MCP-Functions-Server",
    "version": "1.0.0",
    "protocolVersion": MCP_PROTOCOL_VERSION,
    "capabilities": {
        # 서버가 제공하는 기능들을 선언
        "tools": {},      # 도구 실행 지원
        "resources": {},  # 리소스 접근 지원 (미구현)
        "prompts": {}     # 프롬프트 템플릿 지원 (미구현)
    }
}

# ============================================================
# MCP Tools 정의
# ============================================================

MCP_TOOLS = [
    {
        # Tool 1: 현재 시간 조회
        "name": "get_current_time",
        "description": "현재 서버 시간을 UTC 및 KST로 반환합니다",
        "inputSchema": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "타임존 (UTC 또는 KST)",
                    "enum": ["UTC", "KST"]
                }
            }
        }
    },
    {
        # Tool 2: 에코 테스트
        "name": "echo",
        "description": "입력받은 메시지를 그대로 반환합니다 (테스트용)",
        "inputSchema": {
            "type": "object",
            "properties": {
                "message": {
                    "type": "string",
                    "description": "반환할 메시지"
                }
            },
            "required": ["message"]  # 필수 파라미터
        }
    },
    {
        # Tool 3: 계산기
        "name": "calculate",
        "description": "두 숫자의 사칙연산을 수행합니다",
        "inputSchema": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "number",
                    "description": "첫 번째 숫자"
                },
                "b": {
                    "type": "number",
                    "description": "두 번째 숫자"
                },
                "operation": {
                    "type": "string",
                    "description": "연산 종류",
                    "enum": ["add", "subtract", "multiply", "divide"]
                }
            },
            "required": ["a", "b", "operation"]  # 모든 파라미터 필수
        }
    }
]

# ============================================================
# Tool 실행 로직
# ============================================================

def execute_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """
    MCP Tool 실행 함수
    
    Args:
        tool_name: 실행할 도구 이름
        arguments: 도구에 전달할 인자들
    
    Returns:
        MCP 표준 응답 형식의 딕셔너리
        {
            "content": [{"type": "text", "text": "결과"}],
            "isError": False  # 선택적, 에러 시에만
        }
    """
    
    try:
        # Tool 1: 현재 시간 반환
        if tool_name == "get_current_time":
            timezone = arguments.get("timezone", "UTC")
            now = datetime.utcnow()
            
            if timezone == "KST":
                # KST는 UTC+9
                from datetime import timedelta
                now = now + timedelta(hours=9)
                return {
                    "content": [{
                        "type": "text",
                        "text": f"현재 KST 시간: {now.strftime('%Y-%m-%d %H:%M:%S')} KST"
                    }]
                }
            else:
                # 기본값: UTC
                return {
                    "content": [{
                        "type": "text",
                        "text": f"현재 UTC 시간: {now.strftime('%Y-%m-%d %H:%M:%S')} UTC"
                    }]
                }
        
        # Tool 2: 에코 (테스트용)
        elif tool_name == "echo":
            message = arguments.get("message", "")
            return {
                "content": [{
                    "type": "text",
                    "text": f"Echo: {message}"
                }]
            }
        
        # Tool 3: 계산기
        elif tool_name == "calculate":
            # 파라미터 추출 및 형변환
            a = float(arguments.get("a", 0))
            b = float(arguments.get("b", 0))
            operation = arguments.get("operation", "add")
            
            # 연산 수행
            if operation == "add":
                result = a + b
            elif operation == "subtract":
                result = a - b
            elif operation == "multiply":
                result = a * b
            elif operation == "divide":
                # 0으로 나누기 체크
                if b == 0:
                    return {
                        "content": [{
                            "type": "text",
                            "text": "오류: 0으로 나눌 수 없습니다"
                        }],
                        "isError": True
                    }
                result = a / b
            else:
                # 알 수 없는 연산
                return {
                    "content": [{
                        "type": "text",
                        "text": f"오류: 알 수 없는 연산 '{operation}'"
                    }],
                    "isError": True
                }
            
            # 성공 응답
            return {
                "content": [{
                    "type": "text",
                    "text": f"{a} {operation} {b} = {result}"
                }]
            }
        
        # 알 수 없는 도구
        else:
            return {
                "content": [{
                    "type": "text",
                    "text": f"오류: 알 수 없는 도구 '{tool_name}'"
                }],
                "isError": True
            }
    
    except Exception as e:
        # 예외 처리 및 로깅
        logging.error(f"Tool execution error: {str(e)}")
        return {
            "content": [{
                "type": "text",
                "text": f"도구 실행 오류: {str(e)}"
            }],
            "isError": True
        }

# ============================================================
# HTTP Endpoints (Azure Functions Routes)
# ============================================================

@app.route(route="mcp/health", methods=["GET"])
def health_check(req: func.HttpRequest) -> func.HttpResponse:
    """
    헬스 체크 엔드포인트
    
    용도: 서버 상태 확인, 로드 밸런서 health probe
    경로: GET /api/mcp/health
    """
    logging.info("Health check requested")
    
    return func.HttpResponse(
        json.dumps({
            "status": "healthy",
            "timestamp": datetime.utcnow().isoformat(),
            "server": MCP_SERVER_INFO["name"],
            "version": MCP_SERVER_INFO["version"]
        }),
        mimetype="application/json",
        status_code=200
    )

@app.route(route="mcp/info", methods=["GET"])
def server_info(req: func.HttpRequest) -> func.HttpResponse:
    """
    MCP 서버 정보 반환
    
    용도: 클라이언트가 서버 능력 파악
    경로: GET /api/mcp/info
    응답: 서버 메타데이터 및 지원 기능
    """
    logging.info("Server info requested")
    
    return func.HttpResponse(
        json.dumps(MCP_SERVER_INFO),
        mimetype="application/json",
        status_code=200
    )

@app.route(route="mcp/tools", methods=["GET", "POST"])
def tools_endpoint(req: func.HttpRequest) -> func.HttpResponse:
    """
    MCP Tools 엔드포인트
    
    GET: 사용 가능한 도구 목록 반환
    POST: 특정 도구 실행
    
    경로: /api/mcp/tools
    """
    
    if req.method == "GET":
        # Tools 목록 반환
        logging.info("Listing tools")
        return func.HttpResponse(
            json.dumps({
                "tools": MCP_TOOLS
            }),
            mimetype="application/json",
            status_code=200
        )
    
    elif req.method == "POST":
        # Tool 실행
        try:
            # 요청 본문 파싱
            req_body = req.get_json()
            tool_name = req_body.get("name")
            arguments = req_body.get("arguments", {})
            
            logging.info(f"Executing tool: {tool_name} with args: {arguments}")
            
            # Tool 실행
            result = execute_tool(tool_name, arguments)
            
            return func.HttpResponse(
                json.dumps(result),
                mimetype="application/json",
                status_code=200
            )
        
        except ValueError as e:
            # JSON 파싱 실패
            logging.error(f"Invalid request body: {str(e)}")
            return func.HttpResponse(
                json.dumps({
                    "error": "Invalid JSON in request body",
                    "message": str(e)
                }),
                mimetype="application/json",
                status_code=400
            )
        except Exception as e:
            # 기타 예외
            logging.error(f"Tool execution failed: {str(e)}")
            return func.HttpResponse(
                json.dumps({
                    "error": "Tool execution failed",
                    "message": str(e)
                }),
                mimetype="application/json",
                status_code=500
            )

@app.route(route="mcp/messages", methods=["POST"])
def messages_endpoint(req: func.HttpRequest) -> func.HttpResponse:
    """
    MCP Messages 엔드포인트 (JSON-RPC 스타일)
    
    용도: JSON-RPC 2.0 형식의 MCP 메시지 처리
    경로: POST /api/mcp/messages
    
    요청 형식:
    {
        "jsonrpc": "2.0",
        "method": "tools/list" | "tools/call",
        "params": { ... }
    }
    
    응답 형식:
    {
        "jsonrpc": "2.0",
        "result": { ... }
    }
    """
    try:
        # 요청 본문 파싱
        req_body = req.get_json()
        method = req_body.get("method")
        params = req_body.get("params", {})
        
        logging.info(f"MCP Message: method={method}, params={params}")
        
        # 메서드별 처리
        if method == "tools/list":
            # 도구 목록 반환
            response = {
                "jsonrpc": "2.0",
                "result": {
                    "tools": MCP_TOOLS
                }
            }
        elif method == "tools/call":
            # 도구 실행
            tool_name = params.get("name")
            arguments = params.get("arguments", {})
            result = execute_tool(tool_name, arguments)
            response = {
                "jsonrpc": "2.0",
                "result": result
            }
        else:
            # 지원하지 않는 메서드
            response = {
                "jsonrpc": "2.0",
                "error": {
                    "code": -32601,
                    "message": f"Method not found: {method}"
                }
            }
        
        return func.HttpResponse(
            json.dumps(response),
            mimetype="application/json",
            status_code=200
        )
    
    except Exception as e:
        # 내부 오류
        logging.error(f"Messages endpoint error: {str(e)}")
        return func.HttpResponse(
            json.dumps({
                "jsonrpc": "2.0",
                "error": {
                    "code": -32603,
                    "message": f"Internal error: {str(e)}"
                }
            }),
            mimetype="application/json",
            status_code=500
        )

Overwriting src/mcp-function/function_app.py


---

### 💡 참고: FastMCP를 사용한 간단한 구현

위의 코드는 MCP 프로토콜을 직접 구현한 예시입니다. **FastMCP** 라이브러리를 사용하면 더 간결하게 MCP 서버를 구현할 수 있습니다.

#### FastMCP 설치

```bash
pip install fastmcp
```

#### FastMCP + Azure Functions 예시 (`function_app.py`)

```python
import azure.functions as func
from fastmcp import FastMCP

# FastMCP 서버 인스턴스 생성
mcp = FastMCP(
    name="Azure-FastMCP-Server",
    instructions="Azure Functions에서 실행되는 MCP 서버입니다."
)

# MCP Tool 정의: 현재 시간 반환
@mcp.tool()
def get_current_time() -> str:
    """현재 서버 시간을 반환합니다."""
    from datetime import datetime
    return f"현재 UTC 시간: {datetime.utcnow().isoformat()}"

# MCP Tool 정의: 에코
@mcp.tool()
def echo(message: str) -> str:
    """입력 메시지를 그대로 반환합니다."""
    return f"Echo: {message}"

# MCP Tool 정의: 계산기
@mcp.tool()
def calculate(expression: str) -> str:
    """수학 표현식을 계산합니다. (예: '2 + 3 * 4')"""
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"계산 오류: {e}"

# Azure Functions HTTP 트리거 연결
app = func.FunctionApp(http_auth_level=func.AuthLevel.FUNCTION)

@app.route(route="mcp/{*path}", methods=["GET", "POST"])
async def mcp_handler(req: func.HttpRequest) -> func.HttpResponse:
    """MCP 요청을 FastMCP 서버로 위임"""
    return await mcp.handle_azure_function(req)
```

#### 주요 특징

| 항목 | 직접 구현 | FastMCP |
|------|----------|---------|
| **코드량** | 많음 (프로토콜 직접 처리) | 적음 (데코레이터 기반) |
| **Tool 정의** | 딕셔너리로 수동 정의 | `@mcp.tool()` 데코레이터 |
| **타입 추론** | 수동 스키마 작성 | Python 타입힌트에서 자동 추론 |
| **에러 처리** | 직접 구현 | 내장 처리 |

#### 참고 문서

- [FastMCP GitHub](https://github.com/jlowin/fastmcp)
- [FastMCP 공식 문서](https://gofastmcp.com/)
- [Azure Functions Python v2 모델](https://learn.microsoft.com/azure/azure-functions/functions-reference-python)

> ⚠️ **참고**: 이 워크샵에서는 MCP 프로토콜의 이해를 위해 직접 구현 방식을 사용합니다. 실제 프로덕션 환경에서는 FastMCP 같은 라이브러리 사용을 권장합니다.

## 1.2 Bicep으로 인프라 배포

Azure Functions와 APIM 인프라를 **Bicep 템플릿**으로 배포합니다.

### 배포 아키텍처

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         Resource Group (rg-mcp-lab)                         │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  ┌─────────────────────┐     ┌─────────────────────┐                       │
│  │   Storage Account   │     │   Azure Functions   │                       │
│  │   (stmcp-dev-xxx)   │◄────│   (Premium EP1)     │                       │
│  │                     │     │   - Python 3.12     │                       │
│  │   • Blob Storage    │     │   - Managed Identity│                       │
│  │   • Queue Storage   │     │   - MCP 서버 코드    │                       │
│  │   • Table Storage   │     └──────────┬──────────┘                       │
│  │   • Shared Key OFF  │                │                                   │
│  └─────────────────────┘                │ Backend                           │
│                                         ▼                                   │
│                          ┌─────────────────────────────┐                   │
│                          │   API Management (APIM)     │                   │
│                          │   - mcp-api 등록            │                   │
│                          │   - Functions Key 연동      │                   │
│                          │   - Rate Limiting           │                   │
│                          │   - JWT 검증 (Section 4)    │                   │
│                          └─────────────────────────────┘                   │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### 배포되는 리소스

| 리소스 | 이름 규칙 | 설명 |
|--------|----------|------|
| **Storage Account** | `stmcp{env}{suffix}` | Functions 런타임 저장소 (Shared Key 비활성화) |
| **App Service Plan** | `asp-mcp-{env}-{suffix}` | Premium EP1 플랜 |
| **Function App** | `mcp-{env}-func-{suffix}` | Python 3.12, Managed Identity |
| **API Management** | `mcp-{env}-apim-{suffix}` | Consumption SKU |
| **Application Insights** | `appi-mcp-{env}-{suffix}` | 모니터링 및 로깅 |

### 보안 구성 (100% Managed Identity)

이 워크샵은 **Shared Key 없이 100% Managed Identity**로 구성됩니다:

- ✅ **AzureWebJobsStorage**: Managed Identity (RBAC)
- ✅ **Blob/Queue/Table**: Managed Identity (RBAC)
- ✅ **Azure Files**: 미사용 (Premium Plan EP1)
- ✅ **WEBSITE_CONTENTAZUREFILECONNECTIONSTRING**: 미설정

### RBAC 역할 자동 할당

| 역할 | 용도 |
|------|------|
| Storage Blob Data Owner | 코드 저장 |
| Storage Account Contributor | 관리 |
| Storage Queue Data Contributor | 런타임 |
| Storage Table Data Contributor | 런타임 |

### 예상 소요 시간

| 단계 | 소요 시간 |
|------|----------|
| Storage Account 생성 | ~1분 |
| Bicep 배포 | ~10-15분 |
| RBAC 전파 대기 | ~1분 |
| Functions 코드 배포 | ~2-3분 |
| **총 소요 시간** | **~15-20분** |

---
<a id='1.2.1'></a>
### 1.2.1 Storage Account 사전 생성

Functions App이 사용할 **Storage Account**를 먼저 생성합니다.

**핵심 설정:**
- `--allow-shared-key-access false`: Shared Key 완전 비활성화
- `--https-only true`: HTTPS 전용
- `--min-tls-version TLS1_2`: TLS 1.2 이상

> 💡 **Premium Plan에서 Azure Files 미사용 시 100% Managed Identity 가능**
> - `WEBSITE_CONTENTAZUREFILECONNECTIONSTRING`을 설정하지 않으면 Azure Files를 사용하지 않음
> - Shared Key가 불필요하므로 완전히 비활성화 가능

In [5]:
import subprocess
import json

# Bicep 배포를 위한 Storage Account 생성
print("=" * 60)
print("📦 1.2.1 Storage Account 사전 생성")
print("=" * 60)
print()

# 고유 suffix 생성
import hashlib
unique_suffix = hashlib.md5(rg_name.encode()).hexdigest()[:6]
storage_account_name = f"stmcp{environment}{unique_suffix}"

print("ℹ️  Premium Plan에서 Azure Files 미사용 시 100% Managed Identity 가능:")
print("   - AzureWebJobsStorage: Managed Identity")
print("   - WEBSITE_CONTENTAZUREFILECONNECTIONSTRING: 미설정")
print("   - Azure Files: 미사용 (Shared Key 불필요)")
print("   - 참조: azure-function-eventhub-cosmosdb.ipynb와 동일한 방식")
print()

# Storage Account 생성 (Shared Key 완전 비활성화)
result = subprocess.run([
    'az', 'storage', 'account', 'create',
    '--name', storage_account_name,
    '--resource-group', rg_name,
    '--location', location,
    '--sku', 'Standard_LRS',
    '--kind', 'StorageV2',
    '--access-tier', 'Hot',
    '--https-only', 'true',
    '--min-tls-version', 'TLS1_2',
    '--allow-blob-public-access', 'false',
    '--allow-shared-key-access', 'false',
    '--default-action', 'Allow',
    '--public-network-access', 'Enabled',
    '--tags', f'Environment={environment}', 'Project=MCP-Workshop', 'ManagedBy=Bicep'
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Storage Account 생성 완료!")
    print()
    
    # WORKSHOP_CONFIG에 저장
    WORKSHOP_CONFIG['storage_account_name'] = storage_account_name
    
    print("📊 Storage Account 정보:")
    print(f"   📦 이름: {storage_account_name}")
    print(f"   📍 위치: {location}")
    print(f"   💾 SKU: Standard_LRS")
    print(f"   🔐 Shared Key Access: Disabled (100% Managed Identity)")
    print(f"   🌐 Network: Allow all")
    print()
    print("💡 핵심 포인트:")
    print("   WEBSITE_CONTENTAZUREFILECONNECTIONSTRING을 설정하지 않으면")
    print("   Azure Files를 사용하지 않고, Shared Key가 불필요합니다!")
    print()
    
    # Storage Account가 안정화될 때까지 대기
    print("⏳ Storage Account 설정 안정화 대기 중... (10초)")
    import time
    time.sleep(10)
    print("✅ 안정화 완료")
    
else:
    print(f"❌ Storage Account 생성 실패: {result.stderr}")
    
print()

📦 1.2.1 Storage Account 사전 생성

ℹ️  Premium Plan에서 Azure Files 미사용 시 100% Managed Identity 가능:
   - AzureWebJobsStorage: Managed Identity
   - WEBSITE_CONTENTAZUREFILECONNECTIONSTRING: 미설정
   - Azure Files: 미사용 (Shared Key 불필요)
   - 참조: azure-function-eventhub-cosmosdb.ipynb와 동일한 방식

✅ Storage Account 생성 완료!

📊 Storage Account 정보:
   📦 이름: stmcpdevelopment56e69a
   📍 위치: koreacentral
   💾 SKU: Standard_LRS
   🔐 Shared Key Access: Disabled (100% Managed Identity)
   🌐 Network: Allow all

💡 핵심 포인트:
   WEBSITE_CONTENTAZUREFILECONNECTIONSTRING을 설정하지 않으면
   Azure Files를 사용하지 않고, Shared Key가 불필요합니다!

⏳ Storage Account 설정 안정화 대기 중... (10초)
✅ 안정화 완료



---
<a id='1.2.2'></a>
### 1.2.2 Bicep 템플릿 배포

생성된 Storage Account를 사용하여 **나머지 인프라를 Bicep으로 배포**합니다.

**배포되는 리소스:**
- Azure Functions (Premium EP1, Python 3.12)
- API Management (Consumption SKU)
- Application Insights
- RBAC 역할 할당 (4개)

**배포 파라미터:**
```bash
az deployment group create \
  --template-file bicep/main.bicep \
  --parameters existingStorageAccountName={storage} functionPlanSku=EP1
```

> ⏱️ **소요 시간**: 약 10-15분 (APIM 프로비저닝에 시간 소요)

In [8]:
# Bicep 배포 (Premium Plan EP1 - Azure Files 미사용)
storage_account_name = WORKSHOP_CONFIG.get("storage_account_name", "")

if not storage_account_name:
    print("⚠️ Storage Account가 생성되지 않았습니다.")
    print("   먼저 Storage Account를 생성하세요.")
else:
    print("🚀 Bicep 템플릿으로 인프라 배포 중...")
    print(f"   💾 Storage Account: {storage_account_name}")
    print("   📋 배포 구성:")
    print("      - Azure Functions: Premium Plan (EP1)")
    print("      - 100% Managed Identity (Shared Key 불필요)")
    print("      - Azure Files 미사용 (WEBSITE_CONTENTAZUREFILECONNECTIONSTRING 미설정)")
    print("      - RBAC 역할 4개 자동 할당")
    print("   ⏱️  소요 시간: 약 10-15분")
    print()

    deployment_name = f"mcp-deployment-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

    result = subprocess.run([
        'az', 'deployment', 'group', 'create',
        '--resource-group', rg_name,
        '--template-file', 'bicep/main.bicep',
        '--name', deployment_name,
        '--parameters', 
        f'location={location}', 
        f'existingStorageAccountName={storage_account_name}',
        'functionPlanSku=EP1'
    ], capture_output=True, text=True)

    if result.returncode == 0:
        print("✅ 배포 성공!")
        print()
        
        # 배포 출력 파싱
        import json
        try:
            output = json.loads(result.stdout)
            if 'properties' in output and 'outputs' in output['properties']:
                outputs = output['properties']['outputs']
                
                # Function App 이름 저장
                if 'functionAppName' in outputs:
                    function_app_name = outputs['functionAppName']['value']
                    WORKSHOP_CONFIG['function_app_name'] = function_app_name
                    print(f"📦 Function App: {function_app_name}")
                
                # Function App ID 저장
                if 'functionAppId' in outputs:
                    function_app_id = outputs['functionAppId']['value']
                    WORKSHOP_CONFIG['function_app_id'] = function_app_id
                
                # APIM 이름 저장
                if 'apimName' in outputs:
                    apim_name = outputs['apimName']['value']
                    WORKSHOP_CONFIG['apim_name'] = apim_name
                    print(f"🌐 APIM: {apim_name}")
                
                # APIM Gateway URL
                if 'apimGatewayUrl' in outputs:
                    apim_url = outputs['apimGatewayUrl']['value']
                    WORKSHOP_CONFIG['apim_gateway_url'] = apim_url
                    print(f"🔗 APIM Gateway: {apim_url}")
                
                print()
                print("🔐 보안 구성 (Premium Plan EP1 - Azure Files 미사용):")
                print("   ✅ System-assigned Managed Identity 활성화")
                print("   ✅ Storage Account RBAC 역할 4개 할당:")
                print("      • Storage Blob Data Owner (코드 저장)")
                print("      • Storage Account Contributor (관리)")
                print("      • Storage Queue Data Contributor (런타임)")
                print("      • Storage Table Data Contributor (런타임)")
                print("   ✅ 100% Managed Identity (Shared Key 불필요)")
                print("   ✅ Azure Files 미사용")
                print()
                print("💡 핵심 포인트:")
                print("   WEBSITE_CONTENTAZUREFILECONNECTIONSTRING을 설정하지 않아")
                print("   azure-function-eventhub-cosmosdb.ipynb와 동일한 방식으로 작동!")
                print()
                
                # RBAC 역할 전파 대기 (60초)
                print("⏳ RBAC 역할 할당 전파 대기 중... (60초)")
                import time
                for i in range(6):
                    time.sleep(10)
                    print(f"   ⏳ {(i+1)*10}초 경과...")
                print("✅ RBAC 전파 완료!")
                print()
                
                # Functions App 재시작 (설정 적용)
                if 'functionAppName' in outputs:
                    print("🔄 Functions App 재시작 중...")
                    restart_result = subprocess.run([
                        'az', 'functionapp', 'restart',
                        '--name', function_app_name,
                        '--resource-group', rg_name
                    ], capture_output=True, text=True)
                    
                    if restart_result.returncode == 0:
                        print("✅ Functions App 재시작 완료!")
                        print()
                        
                        # 재시작 후 추가 대기 (설정 적용)
                        print("⏳ 설정 적용 대기 중... (30초)")
                        time.sleep(30)
                        print("✅ 설정 적용 완료!")
                    else:
                        print(f"⚠️ 재시작 실패: {restart_result.stderr}")
                print()
        except Exception as e:
            print(f"⚠️ 출력 파싱 실패: {str(e)}")
            print("   수동으로 리소스 이름을 확인하세요.")
        
        print()
        print("✅ 인프라 배포 완료!")
        print()
        print("🔐 보안 상태 (Premium Plan EP1):")
        print("   ✅ AzureWebJobsStorage: Managed Identity (RBAC)")
        print("   ✅ Blob/Queue/Table: Managed Identity (RBAC)")
        print("   ✅ Azure Files: 미사용 (Shared Key 불필요)")
        print()
        print("▶️ 다음: Functions 코드 배포")
    else:
        print(f"❌ 배포 실패: {result.stderr}")
        print()
        print("💡 수동 배포 방법:")
        print(f"   az deployment group create \\")
        print(f"     --resource-group {rg_name} \\")
        print(f"     --template-file bicep/main.bicep \\")
        print(f"     --parameters existingStorageAccountName={storage_account_name} functionPlanSku=EP1")

🚀 Bicep 템플릿으로 인프라 배포 중...
   💾 Storage Account: stmcpdevelopment56e69a
   📋 배포 구성:
      - Azure Functions: Premium Plan (EP1)
      - 100% Managed Identity (Shared Key 불필요)
      - Azure Files 미사용 (WEBSITE_CONTENTAZUREFILECONNECTIONSTRING 미설정)
      - RBAC 역할 4개 자동 할당
   ⏱️  소요 시간: 약 10-15분

✅ 배포 성공!

📦 Function App: mcp-dev-func-i6ht
🌐 APIM: mcp-dev-apim-i6ht
🔗 APIM Gateway: https://mcp-dev-apim-i6ht.azure-api.net

🔐 보안 구성 (Premium Plan EP1 - Azure Files 미사용):
   ✅ System-assigned Managed Identity 활성화
   ✅ Storage Account RBAC 역할 4개 할당:
      • Storage Blob Data Owner (코드 저장)
      • Storage Account Contributor (관리)
      • Storage Queue Data Contributor (런타임)
      • Storage Table Data Contributor (런타임)
   ✅ 100% Managed Identity (Shared Key 불필요)
   ✅ Azure Files 미사용

💡 핵심 포인트:
   WEBSITE_CONTENTAZUREFILECONNECTIONSTRING을 설정하지 않아
   azure-function-eventhub-cosmosdb.ipynb와 동일한 방식으로 작동!

⏳ RBAC 역할 할당 전파 대기 중... (60초)
   ⏳ 10초 경과...
   ⏳ 20초 경과...
   ⏳ 30초 경과...
   ⏳ 40초 경과...
   ⏳ 50초

---
<a id='1.2.3'></a>
### 1.2.3 Functions 코드 배포

**Azure Functions Core Tools**를 사용하여 MCP 서버 코드를 배포합니다.

**배포 명령:**
```bash
cd src/mcp-function
func azure functionapp publish {function-app-name} --python
```

**사전 요구사항:**
- Azure Functions Core Tools v4 설치
  - macOS: `brew install azure-functions-core-tools@4`
  - Windows: `npm install -g azure-functions-core-tools@4`
- Azure CLI 로그인 완료

> ⏱️ **소요 시간**: 약 2-3분

In [9]:
# Functions 코드 배포 (Premium Plan EP1 - Azure Files 미사용)
import subprocess
import os

function_app_name = WORKSHOP_CONFIG.get("function_app_name", "")

if not function_app_name:
    print("⚠️ Function App이 생성되지 않았습니다.")
    print("   먼저 Bicep 배포를 완료하세요.")
else:
    print("=" * 80)
    print(f"📦 Functions App 코드 배포 (Premium Plan EP1): {function_app_name}")
    print("=" * 80)
    print()
    
    # 1. 프로젝트 디렉토리로 이동
    function_dir = 'src/mcp-function'
    
    if not os.path.exists(function_dir):
        print(f"❌ Function 디렉토리를 찾을 수 없습니다: {function_dir}")
        print("   먼저 Function 코드를 작성하세요.")
    else:
        print(f"📁 Function 디렉토리: {function_dir}")
        print("   ℹ️  Azure Files 미사용 - Shared Key 없이 배포됨")
        print()
        
        # 2. 현재 디렉토리 저장
        original_dir = os.getcwd()
        
        try:
            # 3. Function 디렉토리로 이동
            os.chdir(function_dir)
            
            # 4. func CLI로 배포
            print("⏳ Function 배포 중... (약 2-3분 소요)")
            print()
            
            cmd = f"func azure functionapp publish {function_app_name} --python"
            
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            
            # 출력 표시
            print(result.stdout)
            
            if result.stderr and "error" in result.stderr.lower():
                print(result.stderr)
            
            if result.returncode == 0:
                print()
                print("=" * 80)
                print("🎉 Functions App 배포 완료!")
                print("=" * 80)
                print()
                
                # Function URL 표시
                function_url = f"https://{function_app_name}.azurewebsites.net"
                print(f"🔗 Function URL: {function_url}")
                print(f"📍 Health Check: {function_url}/api/health")
                print()
                
                # APIM Named Value 업데이트 안내
                print("📋 다음 단계:")
                print("   1️⃣ Functions App이 완전히 시작될 때까지 1-2분 대기")
                print("   2️⃣ 다음 셀 실행: APIM Functions Key 업데이트")
                print()
            else:
                print()
                print("=" * 80)
                print("❌ 배포 실패")
                print("=" * 80)
                print()
                
                if result.stderr:
                    print("오류 메시지:")
                    print(result.stderr[:500])
                    print()
                
                print("💡 해결 방법:")
                print()
                print("🔧 방법 1: Azure Functions Core Tools 설치 확인")
                print("   macOS: brew install azure-functions-core-tools@4")
                print("   Windows: npm install -g azure-functions-core-tools@4")
                print()
                print("🔧 방법 2: Azure CLI로 로그인 확인")
                print("   az login")
                print(f"   az account set --subscription {WORKSHOP_CONFIG.get('subscription_id', '')}")
                print()
                print("🔧 방법 3: 수동으로 배포 명령 실행")
                print(f"   cd {function_dir}")
                print(f"   func azure functionapp publish {function_app_name} --python")
                print()
        
        finally:
            # 5. 원래 디렉토리로 복귀
            os.chdir(original_dir)

print()

📦 Functions App 코드 배포 (Premium Plan EP1): mcp-dev-func-i6ht

📁 Function 디렉토리: src/mcp-function
   ℹ️  Azure Files 미사용 - Shared Key 없이 배포됨

⏳ Function 배포 중... (약 2-3분 소요)

Getting site publishing info...
[2025-12-11T00:54:36.524Z] Starting the function app deployment...
Updating Application Settings for Remote build...
Creating archive for current directory...
Performing remote build for functions project.
Deleting the old .python_packages directory
Uploading 4.94 KB [----------------------------------------------------------Remote build in progress, please wait...
Updating submodules.
Preparing deployment for commit id 'e71d5a12-1'.
PreDeployment: context.CleanOutputPath False
PreDeployment: context.OutputPath /home/site/wwwroot
Repository path is /tmp/zipdeploy/extracted
Running oryx build...
Command: oryx build /tmp/zipdeploy/extracted -o /tmp/build/expressbuild --platform python --platform-version 3.12 -i /tmp/8de384feeaa0ead -p packagedir=.python_packages/lib/site-packages
Operatio

---
<a id='1.2.4'></a>
### 1.2.4 APIM Functions Key 업데이트

Functions App이 완전히 시작된 후 **실제 Functions Key를 APIM Named Value에 업데이트**합니다.

**동작 과정:**
1. Functions Host Runtime이 준비될 때까지 대기 (최대 3분)
2. Functions 기본 키 (`functionKeys.default`) 가져오기
3. APIM Named Value `functions-key` 업데이트

**APIM에서 Functions Key 사용:**
```xml
<set-header name="x-functions-key" exists-action="override">
    <value>{{functions-key}}</value>
</set-header>
```

> 💡 **핵심**: APIM이 Functions App에 인증된 요청을 보내기 위해 필요

In [10]:
import subprocess
import json
import time

print("=" * 80)
print("🔑 APIM Functions Key 업데이트")
print("=" * 80)
print()

# APIM 리소스 이름 (Bicep 배포 결과에서 가져오기)
apim_name = WORKSHOP_CONFIG.get("apim_name", "")
if not apim_name:
    print("❌ APIM 이름이 설정되지 않았습니다.")
    print("   먼저 Bicep 배포 셀(1.1)을 실행하세요.")
    raise ValueError("apim_name not found in WORKSHOP_CONFIG")

print(f"📌 APIM 이름: {apim_name}")
print()
print("ℹ️  Functions App Host Runtime이 완전히 시작될 때까지 대기 중...")
print("   (약 2-3분 소요)")
print()

# Functions Host Runtime 준비 대기
max_retries = 15
retry_count = 0
runtime_ready = False

while retry_count < max_retries and not runtime_ready:
    retry_count += 1
    print(f"⏳ 시도 {retry_count}/{max_retries}: Host Runtime 상태 확인 중...")
    
    # Functions 키 가져오기 시도 (Runtime이 준비되면 성공)
    test_result = subprocess.run([
        'az', 'functionapp', 'keys', 'list',
        '--name', function_app_name,
        '--resource-group', rg_name,
        '--query', 'functionKeys.default',
        '-o', 'tsv'
    ], capture_output=True, text=True, timeout=60)
    
    if test_result.returncode == 0 and test_result.stdout.strip():
        print("   ✅ Host Runtime 준비 완료")
        runtime_ready = True
    else:
        if retry_count < max_retries:
            print("   ⏳ 아직 준비 중... 15초 후 재시도")
            time.sleep(15)

if not runtime_ready:
    print()
    print("⚠️  Functions Host Runtime이 완전히 준비되지 않았습니다.")
    print("   몇 분 후 이 셀을 다시 실행하세요.")
    print()
else:
    print()
    
    # Functions 기본 키 가져오기
    print("🔑 Functions 기본 키 가져오는 중...")
    key_result = subprocess.run([
        'az', 'functionapp', 'keys', 'list',
        '--name', function_app_name,
        '--resource-group', rg_name,
        '--query', 'functionKeys.default',
        '-o', 'tsv'
    ], capture_output=True, text=True)
    
    if key_result.returncode != 0:
        print(f"❌ Functions 키 가져오기 실패: {key_result.stderr}")
        print()
        print("💡 해결 방법:")
        print("   1. 몇 분 후 이 셀을 다시 실행하세요")
        print("   2. Azure Portal에서 Functions App 상태 확인")
    else:
        functions_key = key_result.stdout.strip()
        print(f"✅ Functions 키 가져오기 완료")
        print()
        
        # APIM Named Value 업데이트
        print("📝 APIM Named Value 'functions-key' 업데이트 중...")
        update_result = subprocess.run([
            'az', 'apim', 'nv', 'update',
            '--service-name', apim_name,
            '--resource-group', rg_name,
            '--named-value-id', 'functions-key',
            '--secret', 'true',
            '--value', functions_key
        ], capture_output=True, text=True)
        
        if update_result.returncode != 0:
            print(f"❌ Named Value 업데이트 실패: {update_result.stderr}")
            print()
            print("💡 수동 업데이트 방법:")
            print(f"   1. Azure Portal → API Management ({apim_name})")
            print("   2. Named values → functions-key")
            print(f"   3. Value를 다음으로 업데이트: {functions_key[:5]}...")
        else:
            print("✅ APIM Named Value 업데이트 완료!")
            print()
            print("=" * 80)
            print("🎉 APIM과 Functions App 연동 완료!")
            print("=" * 80)
            print()
            print("📋 설정 요약:")
            print(f"   🔐 Functions App: {function_app_name}")
            print(f"   🔐 인증 레벨: FUNCTION")
            print(f"   🌐 APIM: {apim_name}")
            print(f"   🔑 Named Value: functions-key (암호화됨)")
            print()
            print("💡 이제 APIM을 통해 Functions App에 안전하게 접근할 수 있습니다!")
            print()
            print("▶️ 다음: MCP 서버 테스트 (Section 1.4)")

print()

🔑 APIM Functions Key 업데이트

📌 APIM 이름: mcp-dev-apim-i6ht

ℹ️  Functions App Host Runtime이 완전히 시작될 때까지 대기 중...
   (약 2-3분 소요)

⏳ 시도 1/15: Host Runtime 상태 확인 중...
   ✅ Host Runtime 준비 완료

🔑 Functions 기본 키 가져오는 중...
✅ Functions 키 가져오기 완료

📝 APIM Named Value 'functions-key' 업데이트 중...
✅ APIM Named Value 업데이트 완료!

🎉 APIM과 Functions App 연동 완료!

📋 설정 요약:
   🔐 Functions App: mcp-dev-func-i6ht
   🔐 인증 레벨: FUNCTION
   🌐 APIM: mcp-dev-apim-i6ht
   🔑 Named Value: functions-key (암호화됨)

💡 이제 APIM을 통해 Functions App에 안전하게 접근할 수 있습니다!

▶️ 다음: MCP 서버 테스트 (Section 1.4)



---

### 💡 참고: APIM에서 Managed Identity로 Functions 호출하기

현재 워크샵에서는 **Function Key**를 사용하여 APIM이 Functions App을 호출하지만, 
**Managed Identity**를 사용하면 키 관리 없이 더 안전하게 인증할 수 있습니다.

#### 전제 조건

1. **Functions App에 Azure AD(Entra ID) 인증 활성화**
   - Functions App → Authentication → Add identity provider → Microsoft
   - 인증 수준을 `Anonymous` 또는 `User`로 설정

2. **APIM에 Managed Identity 할당**
   ```bash
   az apim update --name <apim-name> --resource-group <rg-name> --enable-managed-identity true
   ```

3. **Managed Identity에 역할 부여**
   ```bash
   # Functions App의 App ID 확인
   FUNC_APP_ID=$(az ad app list --display-name <function-app-name> --query "[0].appId" -o tsv)
   
   # APIM Managed Identity에 역할 부여
   az role assignment create \
       --assignee <apim-managed-identity-object-id> \
       --role "Azure Functions Invoker" \
       --scope /subscriptions/<subscription-id>/resourceGroups/<rg-name>/providers/Microsoft.Web/sites/<function-app-name>
   ```

#### APIM 정책 예시

```xml
<policies>
    <inbound>
        <base />
        <!-- Managed Identity로 Azure AD 토큰 발급 -->
        <authentication-managed-identity resource="api://<functions-app-id>" />
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
    </outbound>
</policies>
```

#### Function Key vs Managed Identity 비교

| 항목 | Function Key | Managed Identity |
|------|-------------|------------------|
| **키 관리** | 수동 갱신/교체 필요 | ❌ 키 없음 |
| **보안** | 키 노출 위험 | ✅ Azure AD 기반 |
| **설정 복잡도** | 🟢 낮음 | 🟡 중간 |
| **감사 추적** | 제한적 | ✅ Azure AD 로그 |
| **토큰 만료** | 수동 | ✅ 자동 |

#### 참고 문서

- [APIM Managed Identity로 백엔드 인증](https://learn.microsoft.com/azure/api-management/authentication-managed-identity-policy)
- [Azure Functions에 Azure AD 인증 구성](https://learn.microsoft.com/azure/app-service/configure-authentication-provider-aad)

> ⚠️ **워크샵 참고**: 이 워크샵에서는 설정의 단순화를 위해 Function Key 방식을 사용합니다. 
> 프로덕션 환경에서는 Managed Identity 사용을 권장합니다.

#### 1.3.1 배포 패키지 검증

배포된 Functions를 테스트합니다:

In [11]:
# 배포 검증 (Function Key 사용)
import subprocess
import requests
import time

function_app_name = WORKSHOP_CONFIG.get("function_app_name", "")

if not function_app_name:
    print("⚠️ Function App 이름이 설정되지 않았습니다.")
else:
    print("=" * 60)
    print("🔍 Azure Functions 배포 검증")
    print("=" * 60)
    print()
    
    # Function Key 가져오기 (인증 레벨이 FUNCTION이므로 필요)
    print("🔑 Function Key 가져오는 중...")
    key_result = subprocess.run([
        'az', 'functionapp', 'keys', 'list',
        '--name', function_app_name,
        '--resource-group', rg_name,
        '--query', 'functionKeys.default',
        '-o', 'tsv'
    ], capture_output=True, text=True)
    
    if key_result.returncode != 0 or not key_result.stdout.strip():
        print(f"❌ Function Key 가져오기 실패")
        print("   Functions App이 아직 준비되지 않았을 수 있습니다.")
        print("   잠시 후 다시 시도하세요.")
    else:
        function_key = key_result.stdout.strip()
        print("✅ Function Key 획득 완료")
        print()
        
        # Health Check URL (올바른 라우트: /api/mcp/health)
        function_url = f"https://{function_app_name}.azurewebsites.net"
        health_url = f"{function_url}/api/mcp/health?code={function_key}"
        
        print(f"🔍 Health Check 테스트: {function_url}/api/mcp/health")
        print()
        
        # 배포 후 Functions가 시작될 때까지 대기
        print("⏳ Functions App이 시작될 때까지 대기 중... (최대 60초)")
        
        max_retries = 12  # 60초 (5초 * 12회)
        success = False
        for i in range(max_retries):
            try:
                response = requests.get(health_url, timeout=10)
                if response.status_code == 200:
                    print(f"   ✅ Health Check 성공! ({i*5}초 경과)")
                    print()
                    print("📊 응답 데이터:")
                    print(response.json())
                    print()
                    
                    # Function Key를 WORKSHOP_CONFIG에 저장
                    WORKSHOP_CONFIG['function_key'] = function_key
                    WORKSHOP_CONFIG['function_app_url'] = function_url
                    
                    print("=" * 60)
                    print("✅ Azure Functions 배포 및 검증 완료!")
                    print("=" * 60)
                    success = True
                    break
                elif response.status_code == 401:
                    print(f"   ❌ 인증 실패 (401) - Function Key 확인 필요")
                    break
            except Exception as e:
                if i < max_retries - 1:
                    print(f"   ⏳ 재시도 중... ({i+1}/{max_retries})")
                    time.sleep(5)
        
        if not success:
            print()
            print("❌ Health Check 실패")
            print()
            print("💡 수동 테스트 방법:")
            print(f"   curl \"{function_url}/api/mcp/health?code={function_key[:5]}...\"")
            print()
            print("💡 Azure Portal에서 Functions App 로그를 확인하세요:")
            print(f"   https://portal.azure.com/#resource{WORKSHOP_CONFIG.get('function_app_id', '')}/logStream")

🔍 Azure Functions 배포 검증

🔑 Function Key 가져오는 중...
✅ Function Key 획득 완료

🔍 Health Check 테스트: https://mcp-dev-func-i6ht.azurewebsites.net/api/mcp/health

⏳ Functions App이 시작될 때까지 대기 중... (최대 60초)
   ✅ Health Check 성공! (0초 경과)

📊 응답 데이터:
{'status': 'healthy', 'timestamp': '2025-12-11T01:05:36.603143', 'server': 'Azure-MCP-Functions-Server', 'version': '1.0.0'}

✅ Azure Functions 배포 및 검증 완료!


## 1.4 MCP 서버 테스트

배포된 Functions MCP 서버를 테스트합니다.

In [12]:
# MCP 서버 테스트 (Function Key 사용)
import requests

function_url = WORKSHOP_CONFIG.get("function_app_url", "")
function_key = WORKSHOP_CONFIG.get("function_key", "")

if not function_url or not function_key:
    print("⚠️ Function URL 또는 Function Key가 설정되지 않았습니다.")
    print("   먼저 이전 셀(배포 검증)을 실행하세요.")
else:
    print("=" * 60)
    print("🧪 MCP 서버 테스트")
    print("=" * 60)
    print(f"📌 Function URL: {function_url}")
    print()
    
    all_passed = True
    
    try:
        # Health Check
        print("🧪 테스트 1: Health Check")
        response = requests.get(f"{function_url}/api/mcp/health?code={function_key}", timeout=10)
        if response.status_code == 200:
            print(f"   ✅ {response.json()}")
        else:
            print(f"   ❌ 상태 코드: {response.status_code}")
            all_passed = False
        
        # List Tools
        print("\n🧪 테스트 2: List Tools")
        response = requests.get(f"{function_url}/api/mcp/tools?code={function_key}", timeout=10)
        if response.status_code == 200:
            tools = response.json().get("tools", [])
            print(f"   ✅ {len(tools)}개 도구 발견:")
            for tool in tools:
                print(f"      - {tool['name']}: {tool['description']}")
        else:
            print(f"   ❌ 상태 코드: {response.status_code}")
            all_passed = False
        
        # Call Tool: Echo
        print("\n🧪 테스트 3: Echo Tool")
        payload = {
            "name": "echo",
            "arguments": {"message": "Hello from Workshop!"}
        }
        response = requests.post(
            f"{function_url}/api/mcp/tools?code={function_key}",
            json=payload,
            timeout=10
        )
        if response.status_code == 200:
            result = response.json()
            print(f"   ✅ {result['content'][0]['text']}")
        else:
            print(f"   ❌ 상태 코드: {response.status_code}")
            all_passed = False
        
        print()
        print("=" * 60)
        if all_passed:
            print("✅ 모든 테스트 통과!")
        else:
            print("❌ 일부 테스트 실패 - 위 결과를 확인하세요.")
        print("=" * 60)
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 테스트 실패: {e}")
        print("   Functions가 배포되었는지 확인하세요.")

🧪 MCP 서버 테스트
📌 Function URL: https://mcp-dev-func-i6ht.azurewebsites.net

🧪 테스트 1: Health Check
   ✅ {'status': 'healthy', 'timestamp': '2025-12-11T01:05:42.677111', 'server': 'Azure-MCP-Functions-Server', 'version': '1.0.0'}

🧪 테스트 2: List Tools
   ✅ 3개 도구 발견:
      - get_current_time: 현재 서버 시간을 UTC 및 KST로 반환합니다
      - echo: 입력받은 메시지를 그대로 반환합니다 (테스트용)
      - calculate: 두 숫자의 사칙연산을 수행합니다

🧪 테스트 3: Echo Tool
   ✅ Echo: Hello from Workshop!

✅ 모든 테스트 통과!


<a id='2'></a>
# 2️⃣ APIM으로 엔터프라이즈 게이트웨이 구성

Azure API Management를 MCP 서버의 게이트웨이로 설정합니다.

## 학습 목표
- APIM API 생성 및 Backend 연결
- 정책 (Policies) 적용
- Rate Limiting 및 CORS 설정

## 2.1 APIM 기본 정책 적용

APIM에 기본 정책을 적용합니다 (CORS, Rate Limiting, 로깅).

In [13]:
# APIM 정책 파일 읽기
with open("apim-policies/base-policy.xml", 'r', encoding='utf-8') as f:
    base_policy = f.read()

print("📋 APIM 기본 정책:")
print("=" * 80)
print(base_policy)  # 전체 출력
print("=" * 80)
print()
print("💡 이 정책은 다음을 포함합니다:")
print("   ✅ CORS 설정")
print("   ✅ Rate Limiting (100 calls/분)")
print("   ✅ 요청/응답 로깅")
print("   ✅ MCP 프로토콜 헤더 추가")
print("   ✅ 에러 응답 표준화")
print()
print("🔧 적용 방법:")
print("   1. Azure Portal에서 APIM 열기")
print("   2. APIs > MCP API 선택")
print("   3. 'All operations' 선택")
print("   4. Inbound processing > Policy code editor")
print("   5. 위 XML 내용 붙여넣기")

📋 APIM 기본 정책:
<policies>
    <inbound>
        <base />
        <!-- CORS 설정 -->
        <cors allow-credentials="true">
            <allowed-origins>
                <origin>*</origin>
            </allowed-origins>
            <allowed-methods>
                <method>GET</method>
                <method>POST</method>
                <method>OPTIONS</method>
            </allowed-methods>
            <allowed-headers>
                <header>*</header>
            </allowed-headers>
            <expose-headers>
                <header>*</header>
            </expose-headers>
        </cors>
        
        <!-- Rate Limiting -->
        <rate-limit-by-key calls="100" 
                           renewal-period="60" 
                           counter-key="@(context.Request.IpAddress)" />
        
        <!-- Content-Type 검증 -->
        <choose>
            <when condition="@(context.Request.Method == "POST")">
                <set-header name="Content-Type" exists-action="override">

## 2.2 APIM을 통한 MCP 서버 테스트

APIM Gateway를 통해 MCP 서버를 호출합니다.

In [15]:
# APIM Gateway를 통한 테스트 (API 정책 + Named Value 업데이트 + Subscription Key 자동 획득)
import subprocess
import requests
import json
import time

apim_url = WORKSHOP_CONFIG.get("apim_gateway_url", "")
apim_name = WORKSHOP_CONFIG.get("apim_name", "")
subscription_id = WORKSHOP_CONFIG.get("subscription_id", "")
function_app_name = WORKSHOP_CONFIG.get("function_app_name", "")

if not apim_url or not apim_name:
    print("⚠️ APIM Gateway URL 또는 APIM 이름이 설정되지 않았습니다.")
    print("   먼저 Bicep 배포 셀을 실행하세요.")
else:
    print("=" * 60)
    print("🔧 APIM 설정 및 테스트")
    print("=" * 60)
    print()
    print(f"📌 APIM: {apim_name}")
    print(f"📌 Functions App: {function_app_name}")
    print(f"📌 Resource Group: {rg_name}")
    print()
    
    # Step 1: Functions Key 가져오기
    print("1️⃣ Functions Key 가져오기...")
    func_key_result = subprocess.run([
        'az', 'functionapp', 'keys', 'list',
        '--name', function_app_name,
        '--resource-group', rg_name,
        '--query', 'functionKeys.default',
        '-o', 'tsv'
    ], capture_output=True, text=True)
    
    if func_key_result.returncode != 0 or not func_key_result.stdout.strip():
        print(f"   ❌ Functions Key 가져오기 실패: {func_key_result.stderr}")
    else:
        functions_key = func_key_result.stdout.strip()
        print(f"   ✅ Functions Key 획득: {functions_key[:5]}...")
        WORKSHOP_CONFIG['function_key'] = functions_key
        
        # Step 2: APIM Named Value 'functions-key' 업데이트
        print()
        print("2️⃣ APIM Named Value 'functions-key' 업데이트...")
        
        update_nv_url = (
            f"https://management.azure.com/subscriptions/{subscription_id}"
            f"/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement"
            f"/service/{apim_name}/namedValues/functions-key?api-version=2024-05-01"
        )
        
        nv_body = json.dumps({
            "properties": {
                "displayName": "functions-key",
                "value": functions_key,
                "secret": True
            }
        })
        
        update_nv_result = subprocess.run([
            'az', 'rest', '--method', 'patch',
            '--url', update_nv_url,
            '--body', nv_body,
            '-o', 'none'
        ], capture_output=True, text=True)
        
        if update_nv_result.returncode == 0:
            print("   ✅ Named Value 업데이트 완료!")
        else:
            print(f"   ❌ Named Value 업데이트 실패: {update_nv_result.stderr[:200]}")
        
        # Step 2.5: Backend URL 수정 (/api → /api/mcp)
        print()
        print("2.5️⃣ Backend URL 수정 (경로 추가)...")
        
        backend_url = (
            f"https://management.azure.com/subscriptions/{subscription_id}"
            f"/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement"
            f"/service/{apim_name}/backends/mcp-functions-backend?api-version=2024-05-01"
        )
        
        backend_body = json.dumps({
            "properties": {
                "url": f"https://{function_app_name}.azurewebsites.net/api/mcp",
                "protocol": "http",
                "credentials": {
                    "header": {
                        "x-functions-key": ["{{functions-key}}"]
                    }
                }
            }
        })
        
        backend_result = subprocess.run([
            'az', 'rest', '--method', 'patch',
            '--url', backend_url,
            '--body', backend_body,
            '-o', 'none'
        ], capture_output=True, text=True)
        
        if backend_result.returncode == 0:
            print("   ✅ Backend URL 수정 완료! (/api → /api/mcp)")
        else:
            print(f"   ❌ Backend URL 수정 실패: {backend_result.stderr[:200]}")
        
        # Step 3: API 정책 추가 (Backend 사용하도록 설정)
        print()
        print("3️⃣ APIM API 정책 추가 (Backend 연결)...")
        
        policy_url = (
            f"https://management.azure.com/subscriptions/{subscription_id}"
            f"/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement"
            f"/service/{apim_name}/apis/mcp-api/policies/policy?api-version=2024-05-01"
        )
        
        # API 정책 - Backend 사용 + MCP 헤더 추가
        # Backend URL이 이제 /api/mcp로 설정되어 있으므로 rewrite 불필요
        api_policy = '''<policies>
    <inbound>
        <base />
        <set-backend-service backend-id="mcp-functions-backend" />
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <set-header name="X-MCP-Protocol-Version" exists-action="override">
            <value>2024-11-05</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>'''
        
        policy_body = json.dumps({
            "properties": {
                "format": "xml",
                "value": api_policy
            }
        })
        
        policy_result = subprocess.run([
            'az', 'rest', '--method', 'put',
            '--url', policy_url,
            '--body', policy_body,
            '-o', 'none'
        ], capture_output=True, text=True)
        
        if policy_result.returncode == 0:
            print("   ✅ API 정책 추가 완료!")
        else:
            print(f"   ❌ API 정책 추가 실패: {policy_result.stderr[:200]}")
        
        # Step 4: APIM Subscription Key 가져오기
        print()
        print("4️⃣ APIM Subscription Key 가져오기...")
        
        list_secrets_url = (
            f"https://management.azure.com/subscriptions/{subscription_id}"
            f"/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement"
            f"/service/{apim_name}/subscriptions/master/listSecrets?api-version=2024-05-01"
        )
        
        key_result = subprocess.run([
            'az', 'rest', '--method', 'post',
            '--url', list_secrets_url,
            '--query', 'primaryKey',
            '-o', 'tsv'
        ], capture_output=True, text=True)
        
        if key_result.returncode == 0 and key_result.stdout.strip():
            subscription_key = key_result.stdout.strip()
            WORKSHOP_CONFIG['apim_subscription_key'] = subscription_key
            print(f"   ✅ Subscription Key 획득: {subscription_key[:5]}...")
            
            # Step 5: 정책 적용 대기 후 테스트
            print()
            print("⏳ 정책 적용 대기 중... (3초)")
            time.sleep(3)
            
            print()
            print("=" * 60)
            print("🧪 APIM Gateway를 통한 테스트")
            print("=" * 60)
            print()
            
            headers = {"Ocp-Apim-Subscription-Key": subscription_key}
            
            try:
                print(f"🔍 Health Check: {apim_url}/mcp/health")
                response = requests.get(f"{apim_url}/mcp/health", headers=headers, timeout=15)
                print(f"   상태 코드: {response.status_code}")
                
                if response.status_code == 200:
                    print(f"   ✅ {response.json()}")
                    print(f"   헤더: X-MCP-Protocol-Version = {response.headers.get('X-MCP-Protocol-Version', 'N/A')}")
                    print()
                    print("✅ APIM Gateway 테스트 성공!")
                elif response.status_code == 401:
                    print("   ❌ 401 Unauthorized - Backend 인증 실패")
                    print("   정책/Named Value 적용에 시간이 걸릴 수 있습니다.")
                    print("   10초 후 다시 시도합니다...")
                    time.sleep(10)
                    response = requests.get(f"{apim_url}/mcp/health", headers=headers, timeout=15)
                    if response.status_code == 200:
                        print(f"   ✅ 재시도 성공: {response.json()}")
                    else:
                        print(f"   ❌ 재시도 실패: {response.status_code}")
                else:
                    print(f"   ❌ 응답: {response.text[:200]}")
                    
            except Exception as e:
                print(f"❌ 오류: {e}")
        else:
            print(f"   ❌ Subscription Key 가져오기 실패: {key_result.stderr[:200]}")

🔧 APIM 설정 및 테스트

📌 APIM: mcp-dev-apim-i6ht
📌 Functions App: mcp-dev-func-i6ht
📌 Resource Group: rg-mcp-lab

1️⃣ Functions Key 가져오기...
   ✅ Functions Key 획득: x-QWXw5YReKtPMF...

2️⃣ APIM Named Value 'functions-key' 업데이트...
   ✅ Named Value 업데이트 완료!

2.5️⃣ Backend URL 수정 (경로 추가)...
   ✅ Backend URL 수정 완료! (/api → /api/mcp)

3️⃣ APIM API 정책 추가 (Backend 연결)...
   ✅ API 정책 추가 완료!

4️⃣ APIM Subscription Key 가져오기...
   ✅ Subscription Key 획득: 90c98e4841...

⏳ 정책 적용 대기 중... (3초)

🧪 APIM Gateway를 통한 테스트

🔍 Health Check: https://mcp-dev-apim-i6ht.azure-api.net/mcp/health
   상태 코드: 200
   ✅ {'status': 'healthy', 'timestamp': '2025-12-11T01:06:36.575538', 'server': 'Azure-MCP-Functions-Server', 'version': '1.0.0'}
   헤더: X-MCP-Protocol-Version = 2024-11-05

✅ APIM Gateway 테스트 성공!


<a id='3'></a>
# 3️⃣ 외부 REST API를 MCP 서버로 변환

기존 REST API를 APIM 정책을 통해 MCP 형식으로 변환하여 노출합니다.

## 학습 목표
- REST API를 APIM Backend로 등록
- 요청/응답 변환 정책 작성
- JSONPlaceholder API 예제 구현

> 💡 **JSONPlaceholder**는 무료 Fake REST API로, API 키 없이 테스트할 수 있습니다.
> - URL: https://jsonplaceholder.typicode.com
> - 엔드포인트: `/todos`, `/posts`, `/users` 등

## 3.1 요청 변환 정책 검토

MCP 요청을 REST API 요청으로 변환하는 정책을 살펴봅니다.

> ℹ️ 이 파일은 참조용 템플릿입니다. 실제 적용은 3.4에서 인라인 정책으로 진행합니다.

In [16]:
# 요청 변환 정책 (참조용)
with open("apim-policies/transform-request-policy.xml", 'r', encoding='utf-8') as f:
    request_policy = f.read()

print("📝 요청 변환 정책 (MCP → REST) - 참조용 템플릿:")
print("=" * 80)
print(request_policy)
print("=" * 80)
print()
print("💡 JSONPlaceholder Todos API 변환 과정:")
print("   1. MCP Tool Call 수신:")
print('      {"name": "list_todos", "arguments": {}}')
print()
print("   2. Backend URL 설정:")
print("      → jsonplaceholder-backend (https://jsonplaceholder.typicode.com)")
print()
print("   3. REST API 요청으로 변환:")
print("      → GET /todos")
print()
print("   4. HTTP Method 변경: POST → GET")
print("   5. Body 제거 (GET 요청이므로)")

📝 요청 변환 정책 (MCP → REST) - 참조용 템플릿:
<policies>
    <inbound>
        <base />
        
        <!-- MCP 요청을 REST API 요청으로 변환 -->
        
        <!-- MCP 요청 파싱 -->
        <set-variable name="mcp-tool-name" value="@{
            var body = context.Request.Body.As&lt;JObject&gt;(preserveContent: true);
            var toolName = body?[&quot;params&quot;]?[&quot;name&quot;]?.ToString() ?? body?[&quot;name&quot;]?.ToString() ?? &quot;&quot;;
            return toolName;
        }" />
        
        <set-variable name="mcp-arguments" value="@{
            var body = context.Request.Body.As&lt;JObject&gt;(preserveContent: true);
            var args = body?[&quot;params&quot;]?[&quot;arguments&quot;]?.ToString() ?? body?[&quot;arguments&quot;]?.ToString() ?? &quot;{}&quot;;
            return args;
        }" />
        
        <!-- JSONPlaceholder Todos API 변환 -->
        <choose>
            <!-- list_todos: 전체 할 일 목록 조회 -->
            <when condition="@(context.Variables.GetValueOrDe

## 3.2 응답 변환 정책 검토

REST API 응답을 MCP 형식으로 변환하는 정책을 살펴봅니다.

> ℹ️ 이 파일은 참조용 템플릿입니다. 실제 적용은 3.4에서 인라인 정책으로 진행합니다.

In [17]:
# 응답 변환 정책 (참조용)
with open("apim-policies/transform-response-policy.xml", 'r', encoding='utf-8') as f:
    response_policy = f.read()

print("📝 응답 변환 정책 (REST → MCP) - 참조용 템플릿:")
print("=" * 80)
print(response_policy)
print("=" * 80)
print()
print("💡 JSONPlaceholder Todos API 응답 변환 과정:")
print("   1. JSONPlaceholder 응답 수신:")
print('      [{"userId": 1, "id": 1, "title": "delectus aut...", "completed": false}, ...]')
print()
print("   2. MCP 형식으로 변환:")
print('      {')
print('        "content": [{')
print('          "type": "text",')
print('          "text": "할 일 목록 (처음 5개):\\n[미완료] [1] delectus aut...\\n..."')
print('        }]')
print('      }')
print()
print("   3. Content-Type 설정: application/json")
print("   4. 에러 발생 시 MCP 에러 형식으로 변환")

📝 응답 변환 정책 (REST → MCP) - 참조용 템플릿:
<policies>
    <inbound>
        <base />
    </inbound>
    
    <backend>
        <base />
    </backend>
    
    <outbound>
        <base />
        
        <!-- REST API 응답을 MCP 포맷으로 변환 -->
        
        <!-- 원본 응답 저장 -->
        <set-variable name="original-response" value="@(context.Response.Body.As&lt;string&gt;(preserveContent: true))" />
        
        <!-- MCP 응답 포맷으로 변환 -->
        <set-body>@{
            var originalBody = context.Variables.GetValueOrDefault&lt;string&gt;("original-response");
            
            try {
                // JSONPlaceholder Todos 응답 처리
                // 배열인 경우: 할 일 목록 (list_todos)
                // 객체인 경우: 단일 할 일 (get_todo)
                
                if (originalBody.TrimStart().StartsWith("[")) {
                    // 배열 응답: 할 일 목록
                    var todos = JArray.Parse(originalBody);
                    var sb = new System.Text.StringBuilder();
                    sb.AppendLine("할

## 3.3 JSONPlaceholder API를 MCP로 변환

실제 REST API(JSONPlaceholder)를 APIM에 등록하고 MCP 형식으로 변환하는 과정을 실습합니다.

> 💡 **JSONPlaceholder**는 무료 Fake REST API로, API 키 없이 사용 가능합니다.  
> - https://jsonplaceholder.typicode.com

In [18]:
# JSONPlaceholder API를 APIM에 Backend로 등록
import subprocess
import json
import requests

print("🔗 JSONPlaceholder API를 APIM Backend로 등록")
print("=" * 80)

apim_name = WORKSHOP_CONFIG.get("apim_name", "")
rg_name = WORKSHOP_CONFIG.get("resource_group", "")
subscription_id = WORKSHOP_CONFIG.get("subscription_id", "")

# 1. Backend 등록 (JSONPlaceholder)
print()
print("1️⃣ Backend 등록: jsonplaceholder-backend")

backend_name = "jsonplaceholder-backend"
backend_url = "https://jsonplaceholder.typicode.com"

# Azure 액세스 토큰 획득
token_result = subprocess.run([
    'az', 'account', 'get-access-token',
    '--resource', 'https://management.azure.com/',
    '--query', 'accessToken',
    '--output', 'tsv'
], capture_output=True, text=True)

if token_result.returncode != 0:
    print(f"❌ 토큰 획득 실패: {token_result.stderr}")
else:
    access_token = token_result.stdout.strip()
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # Backend 생성
    backend_api_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement/service/{apim_name}/backends/{backend_name}?api-version=2024-05-01"
    
    backend_body = {
        "properties": {
            "url": backend_url,
            "protocol": "http",
            "title": "JSONPlaceholder API",
            "description": "Free fake REST API for testing"
        }
    }
    
    response = requests.put(backend_api_url, headers=headers, json=backend_body)
    
    if response.status_code in [200, 201]:
        print(f"   ✅ Backend 등록 성공!")
        print(f"   URL: {backend_url}")
    else:
        print(f"   ⚠️ Backend 등록 실패: {response.status_code}")
        print(f"   {response.text[:200]}")

# 2. API 생성 (todos-api)
print()
print("2️⃣ API 생성: todos-api")

api_name = "todos-api"
api_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement/service/{apim_name}/apis/{api_name}?api-version=2024-05-01"

api_body = {
    "properties": {
        "displayName": "Todos MCP API",
        "description": "JSONPlaceholder Todos를 MCP 형식으로 제공",
        "path": "todos",
        "protocols": ["https"],
        "subscriptionRequired": True,
        "subscriptionKeyParameterNames": {
            "header": "Ocp-Apim-Subscription-Key",
            "query": "subscription-key"
        }
    }
}

response = requests.put(api_url, headers=headers, json=api_body)

if response.status_code in [200, 201]:
    print(f"   ✅ API 생성 성공!")
    print(f"   Path: /todos")
else:
    print(f"   ⚠️ API 생성 실패: {response.status_code}")
    print(f"   {response.text[:200]}")

# 3. Operation 추가 (list_todos, get_todo)
print()
print("3️⃣ Operation 추가")

operations = [
    {
        "name": "list-todos",
        "displayName": "List Todos (MCP)",
        "method": "POST",
        "urlTemplate": "/tools/call",
        "description": "모든 할 일 목록 조회 (MCP 형식)"
    },
    {
        "name": "get-todo",
        "displayName": "Get Todo by ID",
        "method": "GET", 
        "urlTemplate": "/{id}",
        "description": "ID로 특정 할 일 조회",
        "templateParameters": [
            {
                "name": "id",
                "type": "integer",
                "required": True,
                "description": "Todo ID"
            }
        ]
    }
]

for op in operations:
    op_name = op["name"]
    op_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement/service/{apim_name}/apis/{api_name}/operations/{op_name}?api-version=2024-05-01"
    
    op_body = {
        "properties": {
            "displayName": op["displayName"],
            "method": op["method"],
            "urlTemplate": op["urlTemplate"],
            "description": op.get("description", ""),
            "templateParameters": op.get("templateParameters", [])
        }
    }
    
    response = requests.put(op_url, headers=headers, json=op_body)
    
    if response.status_code in [200, 201]:
        print(f"   ✅ {op['displayName']}: {op['method']} {op['urlTemplate']}")
    else:
        print(f"   ⚠️ {op['displayName']} 실패: {response.status_code}")

print()
print("=" * 80)
print("✅ JSONPlaceholder API가 APIM에 등록되었습니다!")

🔗 JSONPlaceholder API를 APIM Backend로 등록

1️⃣ Backend 등록: jsonplaceholder-backend
   ✅ Backend 등록 성공!
   URL: https://jsonplaceholder.typicode.com

2️⃣ API 생성: todos-api
   ✅ API 생성 성공!
   Path: /todos

3️⃣ Operation 추가
   ✅ List Todos (MCP): POST /tools/call
   ✅ Get Todo by ID: GET /{id}

✅ JSONPlaceholder API가 APIM에 등록되었습니다!


## 3.4 MCP 변환 정책 적용

REST API 응답을 MCP 형식으로 변환하는 정책을 적용합니다.

In [19]:
# MCP 변환 정책 적용
import requests

print("🔄 MCP 변환 정책 적용")
print("=" * 80)

# list-todos Operation에 MCP 변환 정책 적용
print()
print("1️⃣ list-todos Operation에 변환 정책 적용")

# 정책: MCP 요청을 REST 요청으로 변환하고, REST 응답을 MCP 응답으로 변환
# 참고: XML 속성에서 C# 제네릭(<T>)을 사용할 때는 &lt; &gt; 로 이스케이프 필요
mcp_transform_policy = '''<policies>
    <inbound>
        <base />
        <!-- Backend URL 설정 -->
        <set-backend-service backend-id="jsonplaceholder-backend" />
        <!-- REST API 호출로 변환 (GET /todos) -->
        <rewrite-uri template="/todos" copy-unmatched-params="false" />
        <set-method>GET</set-method>
        <set-body />
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <!-- REST 응답을 MCP 형식으로 변환 -->
        <set-body>@{
            var todos = context.Response.Body.As&lt;JArray&gt;();
            var sb = new System.Text.StringBuilder();
            sb.AppendLine("할 일 목록 (처음 5개):");
            sb.AppendLine("-------------------------");
            
            int count = todos.Count &gt; 5 ? 5 : todos.Count;
            for (int i = 0; i &lt; count; i++) {
                var todo = todos[i];
                var status = (bool)todo["completed"] ? "[완료]" : "[미완료]";
                sb.AppendLine(status + " [" + todo["id"] + "] " + todo["title"]);
            }
            
            sb.AppendLine("-------------------------");
            sb.AppendLine("총 " + todos.Count + "개의 할 일이 있습니다.");
            
            var result = new JObject();
            var content = new JArray();
            var textContent = new JObject();
            textContent["type"] = "text";
            textContent["text"] = sb.ToString();
            content.Add(textContent);
            result["content"] = content;
            return result.ToString();
        }</set-body>
        <set-header name="Content-Type" exists-action="override">
            <value>application/json</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
        <set-body>@{
            var result = new JObject();
            result["isError"] = true;
            var content = new JArray();
            var textContent = new JObject();
            textContent["type"] = "text";
            textContent["text"] = "오류: " + context.LastError.Message;
            content.Add(textContent);
            result["content"] = content;
            return result.ToString();
        }</set-body>
    </on-error>
</policies>'''

# Operation 정책 적용
policy_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement/service/{apim_name}/apis/todos-api/operations/list-todos/policies/policy?api-version=2024-05-01"

policy_body = {
    "properties": {
        "format": "xml",
        "value": mcp_transform_policy
    }
}

response = requests.put(policy_url, headers=headers, json=policy_body)

if response.status_code in [200, 201]:
    print("   ✅ MCP 변환 정책이 적용되었습니다!")
    print()
    print("   📌 정책 동작:")
    print("      1. MCP 요청 수신: POST /todos/tools/call")
    print("      2. Backend 호출: GET /todos (JSONPlaceholder)")
    print("      3. 응답 변환: JSON 배열 → MCP content 형식")
else:
    print(f"   ⚠️ 정책 적용 실패: {response.status_code}")
    print(f"   {response.text[:300]}")

# get-todo Operation 정책 적용
print()
print("2️⃣ get-todo Operation에 Backend 정책 적용")

get_todo_policy = '''<policies>
    <inbound>
        <base />
        <set-backend-service backend-id="jsonplaceholder-backend" />
        <rewrite-uri template="@{return &quot;/todos/&quot; + context.Request.MatchedParameters[&quot;id&quot;];}" copy-unmatched-params="false" />
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <!-- REST 응답을 MCP 형식으로 변환 -->
        <set-body>@{
            var todo = context.Response.Body.As&lt;JObject&gt;();
            var status = (bool)todo["completed"] ? "완료" : "미완료";
            var text = "할 일 상세\\n-------------------------\\nID: " + todo["id"] + "\\n제목: " + todo["title"] + "\\n상태: " + status + "\\n담당자 ID: " + todo["userId"];
            
            var result = new JObject();
            var content = new JArray();
            var textContent = new JObject();
            textContent["type"] = "text";
            textContent["text"] = text;
            content.Add(textContent);
            result["content"] = content;
            return result.ToString();
        }</set-body>
        <set-header name="Content-Type" exists-action="override">
            <value>application/json</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>'''

policy_url2 = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement/service/{apim_name}/apis/todos-api/operations/get-todo/policies/policy?api-version=2024-05-01"

policy_body2 = {
    "properties": {
        "format": "xml",
        "value": get_todo_policy
    }
}

response = requests.put(policy_url2, headers=headers, json=policy_body2)

if response.status_code in [200, 201]:
    print("   ✅ get-todo 정책이 적용되었습니다!")
else:
    print(f"   ⚠️ 정책 적용 실패: {response.status_code}")
    print(f"   {response.text[:300]}")

print()
print("=" * 80)
print("✅ MCP 변환 정책 적용 완료!")

🔄 MCP 변환 정책 적용

1️⃣ list-todos Operation에 변환 정책 적용
   ✅ MCP 변환 정책이 적용되었습니다!

   📌 정책 동작:
      1. MCP 요청 수신: POST /todos/tools/call
      2. Backend 호출: GET /todos (JSONPlaceholder)
      3. 응답 변환: JSON 배열 → MCP content 형식

2️⃣ get-todo Operation에 Backend 정책 적용
   ✅ get-todo 정책이 적용되었습니다!

✅ MCP 변환 정책 적용 완료!


## 3.5 MCP 변환 테스트

APIM을 통해 JSONPlaceholder API가 MCP 형식으로 변환되는지 테스트합니다.

In [20]:
# MCP 변환 테스트
import requests
import json
import subprocess

print("🧪 MCP 변환 테스트")
print("=" * 80)

apim_name = WORKSHOP_CONFIG.get("apim_name", "")
apim_url = WORKSHOP_CONFIG.get("apim_gateway_url", f"https://{apim_name}.azure-api.net")

# Subscription Key 가져오기 (WORKSHOP_CONFIG에 없으면 직접 조회)
subscription_key = WORKSHOP_CONFIG.get("subscription_key", "")

if not subscription_key:
    print("📌 Subscription Key 조회 중...")
    subscription_id = WORKSHOP_CONFIG.get("subscription_id", "")
    rg_name = WORKSHOP_CONFIG.get("resource_group", "")
    
    # 액세스 토큰 획득
    token_result = subprocess.run([
        'az', 'account', 'get-access-token',
        '--resource', 'https://management.azure.com/',
        '--query', 'accessToken',
        '--output', 'tsv'
    ], capture_output=True, text=True)
    
    if token_result.returncode == 0:
        access_token = token_result.stdout.strip()
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }
        
        # Master subscription의 키 조회
        list_secrets_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement/service/{apim_name}/subscriptions/master/listSecrets?api-version=2024-05-01"
        
        secrets_response = requests.post(list_secrets_url, headers=headers)
        if secrets_response.status_code == 200:
            subscription_key = secrets_response.json().get("primaryKey", "")
            print(f"   ✅ Subscription Key 획득 완료")
        else:
            print(f"   ⚠️ Subscription Key 조회 실패: {secrets_response.status_code}")
    else:
        print(f"   ⚠️ 토큰 획득 실패")
    print()

# 테스트 1: list_todos (MCP 형식으로 호출)
print()
print("1️⃣ list_todos 테스트 (MCP 형식)")
print("-" * 40)

mcp_request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/call",
    "params": {
        "name": "list_todos",
        "arguments": {}
    }
}

test_url = f"{apim_url}/todos/tools/call"
test_headers = {
    "Content-Type": "application/json",
    "Ocp-Apim-Subscription-Key": subscription_key
}

print(f"   URL: {test_url}")
print(f"   Method: POST")
print(f"   Request: {json.dumps(mcp_request, indent=2)}")
print()

try:
    response = requests.post(test_url, headers=test_headers, json=mcp_request, timeout=30)
    
    print(f"   Status: {response.status_code}")
    print()
    
    if response.status_code == 200:
        result = response.json()
        print("   📤 MCP 응답:")
        print("   " + "-" * 36)
        
        if "content" in result:
            for content in result["content"]:
                if content.get("type") == "text":
                    for line in content["text"].split("\n"):
                        print(f"   {line}")
        else:
            print(f"   {json.dumps(result, indent=2, ensure_ascii=False)}")
        
        print()
        print("   ✅ REST → MCP 변환 성공!")
    else:
        print(f"   ❌ 테스트 실패: {response.text[:200]}")
except Exception as e:
    print(f"   ❌ 오류: {e}")

# 테스트 2: get_todo (특정 할 일 조회)
print()
print("2️⃣ get_todo 테스트 (ID=1)")
print("-" * 40)

test_url2 = f"{apim_url}/todos/1"

print(f"   URL: {test_url2}")
print(f"   Method: GET")
print()

try:
    response = requests.get(test_url2, headers={"Ocp-Apim-Subscription-Key": subscription_key}, timeout=30)
    
    print(f"   Status: {response.status_code}")
    print()
    
    if response.status_code == 200:
        result = response.json()
        print("   📤 MCP 응답:")
        print("   " + "-" * 36)
        
        if "content" in result:
            for content in result["content"]:
                if content.get("type") == "text":
                    for line in content["text"].split("\n"):
                        print(f"   {line}")
        else:
            print(f"   {json.dumps(result, indent=2, ensure_ascii=False)}")
        
        print()
        print("   ✅ REST → MCP 변환 성공!")
    else:
        print(f"   ❌ 테스트 실패: {response.text[:200]}")
except Exception as e:
    print(f"   ❌ 오류: {e}")

print()
print("=" * 80)
print()
print("📊 변환 요약:")
print("   ┌─────────────────────────────────────────────────────┐")
print("   │  JSONPlaceholder REST API  →  MCP 형식 응답        │")
print("   ├─────────────────────────────────────────────────────┤")
print("   │  GET /todos               →  POST /tools/call      │")
print("   │  JSON Array               →  MCP content           │")
print("   │  GET /todos/{id}          →  GET /{id}             │")
print("   │  JSON Object              →  MCP content           │")
print("   └─────────────────────────────────────────────────────┘")

🧪 MCP 변환 테스트
📌 Subscription Key 조회 중...
   ✅ Subscription Key 획득 완료


1️⃣ list_todos 테스트 (MCP 형식)
----------------------------------------
   URL: https://mcp-dev-apim-i6ht.azure-api.net/todos/tools/call
   Method: POST
   Request: {
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": {
    "name": "list_todos",
    "arguments": {}
  }
}

   Status: 200

   📤 MCP 응답:
   ------------------------------------
   할 일 목록 (처음 5개):
   -------------------------
   [미완료] [1] delectus aut autem
   [미완료] [2] quis ut nam facilis et officia qui
   [미완료] [3] fugiat veniam minus
   [완료] [4] et porro tempora
   [미완료] [5] laboriosam mollitia et enim quasi adipisci quia provident illum
   -------------------------
   총 200개의 할 일이 있습니다.
   

   ✅ REST → MCP 변환 성공!

2️⃣ get_todo 테스트 (ID=1)
----------------------------------------
   URL: https://mcp-dev-apim-i6ht.azure-api.net/todos/1
   Method: GET

   Status: 200

   📤 MCP 응답:
   ------------------------------------
   할 일 상세
   -------

## 3.6 (참고) OpenWeatherMap API로 확장하기

JSONPlaceholder 대신 실제 날씨 API를 사용하고 싶다면 **OpenWeatherMap API**를 사용할 수 있습니다.

### 🌤️ OpenWeatherMap 설정 방법

#### 1. API Key 발급
1. [OpenWeatherMap](https://openweathermap.org/api) 가입
2. **Free Plan** 선택 (60 calls/minute, 1,000,000 calls/month)
3. API Key 발급 (발급 후 활성화까지 ~10분 소요)

#### 2. Backend 등록
```python
backend_body = {
    "properties": {
        "url": "https://api.openweathermap.org/data/2.5",
        "protocol": "http",
        "title": "OpenWeatherMap API",
        "description": "Current weather data API"
    }
}
```

#### 3. Named Value에 API Key 저장
```bash
az apim nv create \
  --resource-group $RG_NAME \
  --service-name $APIM_NAME \
  --named-value-id weather-api-key \
  --display-name "Weather API Key" \
  --value "YOUR_API_KEY" \
  --secret true
```

#### 4. 요청 변환 정책
```xml
<policies>
    <inbound>
        <base />
        <set-backend-service backend-id="openweathermap-backend" />
        <!-- MCP 요청에서 city 파라미터 추출 -->
        <set-variable name="city" value="@{
            var body = context.Request.Body.As<JObject>(preserveContent: true);
            var args = body?["params"]?["arguments"];
            return args?["city"]?.ToString() ?? "Seoul";
        }" />
        <!-- REST API 형식으로 변환 -->
        <rewrite-uri template="@("/weather?q=" + context.Variables["city"] + "&appid={{weather-api-key}}&units=metric")" />
        <set-method>GET</set-method>
        <set-body />
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <!-- 응답을 MCP 형식으로 변환 -->
        <set-body>@{
            var weather = context.Response.Body.As<JObject>();
            var name = weather["name"]?.ToString();
            var temp = weather["main"]?["temp"]?.ToString();
            var desc = weather["weather"]?[0]?["description"]?.ToString();
            
            var text = $"🌡️ {name} 날씨\n" +
                      $"온도: {temp}°C\n" +
                      $"상태: {desc}";
            
            var result = new JObject();
            var content = new JArray();
            var textContent = new JObject();
            textContent["type"] = "text";
            textContent["text"] = text;
            content.Add(textContent);
            result["content"] = content;
            return result.ToString();
        }</set-body>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>
```

#### 5. 호출 예시
```bash
curl -X POST "https://{apim-name}.azure-api.net/weather/tools/call" \
  -H "Content-Type: application/json" \
  -H "Ocp-Apim-Subscription-Key: {key}" \
  -d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/call",
    "params": {
      "name": "get_weather",
      "arguments": {"city": "Seoul"}
    }
  }'
```

#### 6. 예상 응답
```json
{
  "content": [{
    "type": "text",
    "text": "🌡️ Seoul 날씨\n온도: 5.2°C\n상태: clear sky"
  }]
}
```

---

### 📊 JSONPlaceholder vs OpenWeatherMap 비교

| 항목 | JSONPlaceholder | OpenWeatherMap |
|------|-----------------|----------------|
| **API Key** | 불필요 | 필요 (무료) |
| **Rate Limit** | 없음 | 60 calls/min |
| **데이터** | 정적 Mock | 실시간 날씨 |
| **용도** | 개발/테스트 | 프로덕션 가능 |
| **난이도** | 쉬움 | 보통 |

> 💡 **권장**: 워크샵에서는 JSONPlaceholder로 먼저 개념을 익힌 후, 실제 프로젝트에서 OpenWeatherMap 같은 실제 API로 확장하세요.

<a id='4'></a>
# 4️⃣ OpenID Connect 기반 인증

Entra ID와 OpenID Connect를 사용하여 MCP 서버에 엔터프라이즈급 인증을 추가합니다.

---

## 🎯 학습 목표

이 섹션을 완료하면 다음을 수행할 수 있습니다:

- ✅ Entra ID에 API 앱과 Client 앱 등록
- ✅ OAuth 2.0 Client Credentials Flow 이해 및 구현
- ✅ APIM `validate-jwt` 정책으로 토큰 검증
- ✅ Bearer 토큰 기반 API 보안 적용

---

## 🏗️ 인증 아키텍처

```
┌─────────────────┐                    ┌─────────────────┐
│   MCP Client    │───── 1. 토큰 요청 ────▶│   Entra ID      │
│   (앱/서비스)    │◀──── 2. JWT 토큰 ──────│   (IdP)         │
└────────┬────────┘                    └─────────────────┘
         │
         │ 3. API 호출 (Authorization: Bearer <token>)
         ▼
┌────────────────────────────────────────────────────────┐
│              Azure API Management                       │
│  ┌────────────────────────────────────────────────┐    │
│  │  validate-jwt 정책                              │    │
│  │  ✅ OpenID Configuration 자동 검색              │    │
│  │  ✅ Audience (aud) 클레임 검증                  │    │
│  │  ✅ Issuer (iss) 클레임 검증                    │    │
│  │  ✅ 토큰 만료 시간 (exp) 검증                   │    │
│  │  ✅ JWKS 기반 서명 검증                         │    │
│  └────────────────────────────────────────────────┘    │
└────────────────────────┬───────────────────────────────┘
                         │ 4. 인증된 요청 전달
                         ▼
                ┌──────────────────┐
                │  Azure Functions │
                │   MCP Server     │
                └──────────────────┘
```

---

## 📋 인증 흐름 단계별 설명

| 단계 | 설명 |
|------|------|
| **1. 토큰 요청** | Client App이 Entra ID에 Client ID + Secret으로 토큰 요청 |
| **2. JWT 발급** | Entra ID가 서명된 JWT Access Token 반환 |
| **3. API 호출** | Client가 `Authorization: Bearer {token}` 헤더와 함께 APIM 호출 |
| **4. 토큰 검증** | APIM이 `validate-jwt` 정책으로 토큰 유효성 검사 |
| **5. Backend 호출** | 검증 성공 시 Azure Functions MCP 서버로 요청 전달 |

---

## 🔑 OAuth 2.0 Client Credentials Flow

이 워크샵에서는 **Client Credentials Flow**를 사용합니다:

### 사용 시나리오
- 서버 간 통신 (Machine-to-Machine, M2M)
- 사용자 없이 앱 자체가 인증
- 백그라운드 서비스, 데몬 앱, CLI 도구

### Flow 비교
| 항목 | Client Credentials Flow | Authorization Code Flow |
|------|------------------------|------------------------|
| **사용자** | 없음 (앱만) | 사용자 로그인 필요 |
| **토큰 내용** | 앱 정보만 | 사용자 + 앱 정보 |
| **용도** | 서버 간 통신 | 웹/모바일 앱 |
| **Refresh Token** | 없음 | 있음 |
| **UI 필요** | 불필요 | 로그인 화면 필요 |

---

## 🔧 생성할 Entra ID 리소스

| 앱 이름 | 역할 | 용도 |
|---------|------|------|
| **MCP-API** | Resource Server | MCP 서버를 나타내는 앱 (보호 대상) |
| **MCP-Client** | Client App | MCP 서버를 호출하는 클라이언트 |

### MCP-API 앱 설정
- **Application ID URI**: `api://{app-id}` (토큰의 `aud` 클레임)
- **OAuth2 Scope**: `mcp.access` (권한 범위 정의)
- **Service Principal**: Enterprise Application으로 등록

### MCP-Client 앱 설정
- **Client Secret**: 앱 인증을 위한 비밀 키 (1년 유효)
- **API Permission**: MCP-API에 대한 `mcp.access` 권한
- **Admin Consent**: 관리자 사전 승인 (Client Credentials에 필수)

## 4.1 Entra ID 앱 등록

아래 코드 셀은 Azure CLI를 사용하여 두 개의 Entra ID 앱을 자동으로 등록합니다.

---

### 📌 등록되는 앱 구성

#### 1️⃣ API 앱 (MCP-API-{suffix})

**목적**: MCP 서버를 나타내는 Resource Server

| 설정 항목 | 값 | 설명 |
|-----------|-----|------|
| Display Name | `MCP-API-{suffix}` | 앱 표시 이름 |
| Sign-in Audience | `AzureADMyOrg` | 현재 테넌트 사용자만 |
| Application ID URI | `api://{app-id}` | 토큰의 `aud` 클레임 값 |
| OAuth2 Scope | `mcp.access` | 정의된 권한 범위 |
| Service Principal | 자동 생성 | Enterprise App으로 표시 |

#### 2️⃣ Client 앱 (MCP-Client-{suffix})

**목적**: MCP 서버를 호출하는 클라이언트 애플리케이션

| 설정 항목 | 값 | 설명 |
|-----------|-----|------|
| Display Name | `MCP-Client-{suffix}` | 앱 표시 이름 |
| Client Secret | 자동 생성 (1년) | `ENTRA_CLIENT_SECRET` 환경변수에 저장 |
| API Permission | `MCP-API/.default` | API 앱에 대한 접근 권한 |
| Admin Consent | 자동 승인 | Client Credentials에 필수 |

---

### 🔐 생성되는 환경 변수

코드 실행 후 다음 환경 변수가 설정됩니다:

| 환경 변수 | 용도 |
|-----------|------|
| `ENTRA_API_APP_ID` | API 앱의 Application (Client) ID |
| `ENTRA_CLIENT_APP_ID` | Client 앱의 Application (Client) ID |
| `ENTRA_CLIENT_SECRET` | Client 앱의 비밀 키 |

---

### ⚠️ 사전 요구사항

- Azure CLI 로그인 완료 (`az login`)
- Entra ID **Application Administrator** 또는 **Global Administrator** 역할
- 앱 등록 권한 (테넌트 설정에서 "Users can register applications" 활성화)

In [21]:
# Entra ID 앱 자동 등록
import subprocess
import json
import os

print("🔐 Entra ID 앱 자동 등록")
print("=" * 80)

tenant_id = WORKSHOP_CONFIG.get("tenant_id", "")
unique_suffix = WORKSHOP_CONFIG.get("unique_suffix", "")

# 1. API 앱 등록 (MCP API)
print()
print("1️⃣ API 앱 등록 (MCP-API)...")

api_app_name = f"MCP-API-{unique_suffix}"
api_app_result = subprocess.run([
    'az', 'ad', 'app', 'create',
    '--display-name', api_app_name,
    '--sign-in-audience', 'AzureADMyOrg',
    '--query', '{appId: appId, id: id}',
    '-o', 'json'
], capture_output=True, text=True)

if api_app_result.returncode == 0:
    api_app_info = json.loads(api_app_result.stdout)
    api_app_id = api_app_info['appId']
    api_object_id = api_app_info['id']
    print(f"   ✅ API App 생성됨: {api_app_id}")
    
    # Application ID URI 설정
    api_uri = f"api://{api_app_id}"
    subprocess.run([
        'az', 'ad', 'app', 'update',
        '--id', api_app_id,
        '--identifier-uris', api_uri
    ], capture_output=True, text=True)
    print(f"   ✅ Application ID URI: {api_uri}")
    
    # Scope 추가 (mcp.access)
    scope_id = subprocess.run([
        'az', 'ad', 'app', 'show',
        '--id', api_app_id,
        '--query', 'id',
        '-o', 'tsv'
    ], capture_output=True, text=True).stdout.strip()
    
    # OAuth2 permission scope 추가 (Graph API 사용)
    import uuid
    scope_guid = str(uuid.uuid4())
    
    scope_manifest = json.dumps({
        "oauth2PermissionScopes": [{
            "adminConsentDescription": "Allow access to MCP API",
            "adminConsentDisplayName": "Access MCP API",
            "id": scope_guid,
            "isEnabled": True,
            "type": "User",
            "userConsentDescription": "Allow access to MCP API",
            "userConsentDisplayName": "Access MCP API",
            "value": "mcp.access"
        }]
    })
    
    scope_result = subprocess.run([
        'az', 'ad', 'app', 'update',
        '--id', api_app_id,
        '--set', f'api={scope_manifest}'
    ], capture_output=True, text=True)
    
    if scope_result.returncode == 0:
        print(f"   ✅ Scope 추가됨: mcp.access")
    
    # API App의 Service Principal 생성
    print("   📌 API App Service Principal 생성 중...")
    api_sp_result = subprocess.run([
        'az', 'ad', 'sp', 'create',
        '--id', api_app_id
    ], capture_output=True, text=True)
    
    if api_sp_result.returncode == 0 or "already exists" in api_sp_result.stderr.lower():
        print("   ✅ API Service Principal 생성됨")
    
    os.environ['ENTRA_API_APP_ID'] = api_app_id
    WORKSHOP_CONFIG['entra_api_app_id'] = api_app_id
    
else:
    print(f"   ❌ API App 생성 실패: {api_app_result.stderr}")
    api_app_id = None

# 2. Client 앱 등록 (MCP Client)
if api_app_id:
    print()
    print("2️⃣ Client 앱 등록 (MCP-Client)...")
    
    client_app_name = f"MCP-Client-{unique_suffix}"
    client_app_result = subprocess.run([
        'az', 'ad', 'app', 'create',
        '--display-name', client_app_name,
        '--sign-in-audience', 'AzureADMyOrg',
        '--query', 'appId',
        '-o', 'tsv'
    ], capture_output=True, text=True)
    
    if client_app_result.returncode == 0:
        client_app_id = client_app_result.stdout.strip()
        print(f"   ✅ Client App 생성됨: {client_app_id}")
        
        # Service Principal 생성 (Enterprise Application)
        print("   📌 Service Principal 생성 중...")
        sp_result = subprocess.run([
            'az', 'ad', 'sp', 'create',
            '--id', client_app_id
        ], capture_output=True, text=True)
        
        if sp_result.returncode == 0 or "already exists" in sp_result.stderr.lower():
            print("   ✅ Service Principal 생성됨")
        else:
            print(f"   ⚠️ Service Principal 생성 경고: {sp_result.stderr[:100]}")
        
        # Client Secret 생성
        secret_result = subprocess.run([
            'az', 'ad', 'app', 'credential', 'reset',
            '--id', client_app_id,
            '--append',
            '--display-name', 'MCP-Client-Secret',
            '--years', '1',
            '--query', 'password',
            '-o', 'tsv'
        ], capture_output=True, text=True)
        
        if secret_result.returncode == 0:
            client_secret = secret_result.stdout.strip()
            print(f"   ✅ Client Secret 생성됨: {client_secret[:5]}...")
            
            os.environ['ENTRA_CLIENT_APP_ID'] = client_app_id
            os.environ['ENTRA_CLIENT_SECRET'] = client_secret
            WORKSHOP_CONFIG['entra_client_app_id'] = client_app_id
            WORKSHOP_CONFIG['entra_client_secret'] = client_secret
        else:
            print(f"   ❌ Client Secret 생성 실패: {secret_result.stderr}")
        
        # API 권한 추가 (MCP-API에 대한 권한)
        print()
        print("3️⃣ API 권한 추가...")
        
        # API App의 scope ID 가져오기
        api_scopes_result = subprocess.run([
            'az', 'ad', 'app', 'show',
            '--id', api_app_id,
            '--query', 'api.oauth2PermissionScopes[0].id',
            '-o', 'tsv'
        ], capture_output=True, text=True)
        
        if api_scopes_result.returncode == 0 and api_scopes_result.stdout.strip():
            scope_id = api_scopes_result.stdout.strip()
            
            # Client App에 API 권한 추가
            permission_result = subprocess.run([
                'az', 'ad', 'app', 'permission', 'add',
                '--id', client_app_id,
                '--api', api_app_id,
                '--api-permissions', f'{scope_id}=Scope'
            ], capture_output=True, text=True)
            
            if permission_result.returncode == 0:
                print(f"   ✅ API 권한 추가됨")
                
                # Admin consent 부여
                print()
                print("4️⃣ Admin Consent 부여...")
                consent_result = subprocess.run([
                    'az', 'ad', 'app', 'permission', 'admin-consent',
                    '--id', client_app_id
                ], capture_output=True, text=True)
                
                if consent_result.returncode == 0:
                    print("   ✅ Admin Consent 완료!")
                else:
                    print(f"   ⚠️ Admin Consent 수동 필요: {consent_result.stderr[:100]}")
            else:
                print(f"   ⚠️ API 권한 추가 실패 (수동 설정 필요): {permission_result.stderr[:100]}")
        
        print()
        print("=" * 80)
        print("✅ Entra ID 설정 완료!")
        print("=" * 80)
        print()
        print(f"📌 Tenant ID: {tenant_id}")
        print(f"📌 API App ID: {api_app_id}")
        print(f"📌 Client App ID: {client_app_id}")
        print(f"📌 Client Secret: {client_secret[:5]}... (환경변수에 저장됨)")
        print()
        print("💡 환경 변수가 자동으로 설정되었습니다:")
        print(f"   ENTRA_API_APP_ID = {api_app_id}")
        print(f"   ENTRA_CLIENT_APP_ID = {client_app_id}")
        print(f"   ENTRA_CLIENT_SECRET = ***")
        
    else:
        print(f"   ❌ Client App 생성 실패: {client_app_result.stderr}")

🔐 Entra ID 앱 자동 등록

1️⃣ API 앱 등록 (MCP-API)...
   ✅ API App 생성됨: 56a56150-850b-4889-8bab-cc775012d4ba
   ✅ Application ID URI: api://56a56150-850b-4889-8bab-cc775012d4ba
   📌 API App Service Principal 생성 중...

2️⃣ Client 앱 등록 (MCP-Client)...
   ✅ Client App 생성됨: 3c1271cc-b9a6-44b3-9ccc-0145f9bdba1e
   📌 Service Principal 생성 중...
   ⚠️ Service Principal 생성 경고: ERROR: The service principal cannot be created, updated, or restored because the service principal n
   ✅ Client Secret 생성됨: gNl8Q~-KjR...

3️⃣ API 권한 추가...
   ✅ API 권한 추가됨

4️⃣ Admin Consent 부여...
   ⚠️ Admin Consent 수동 필요: ERROR: Forbidden({"ClassName":"Microsoft.Portal.Framework.Exceptions.ClientException","Message":"Gra

✅ Entra ID 설정 완료!

📌 Tenant ID: 38bd4dc5-c85c-4efd-a73a-3dfe7d6f632e
📌 API App ID: 56a56150-850b-4889-8bab-cc775012d4ba
📌 Client App ID: 3c1271cc-b9a6-44b3-9ccc-0145f9bdba1e
📌 Client Secret: gNl8Q~-KjR... (환경변수에 저장됨)

💡 환경 변수가 자동으로 설정되었습니다:
   ENTRA_API_APP_ID = 56a56150-850b-4889-8bab-cc775012d4ba
   ENTRA_CLI

---

## 📌 (선택) 개별 사용자/그룹 단위 권한 할당

위의 4.1 단계에서는 `admin-consent`를 통해 **조직 전체에 권한을 승인**했습니다. 
더 세밀한 접근 제어가 필요한 경우, 특정 사용자나 그룹에만 권한을 부여할 수 있습니다.

---

### 🎯 Admin Consent vs 개별 할당 비교

| 방식 | 적용 범위 | 사용 시점 | 보안 수준 |
|------|----------|----------|----------|
| **Admin Consent** | 조직 전체 | 개발/테스트, 내부 서비스 | 낮음 |
| **개별 할당** | 지정된 사용자/그룹만 | 프로덕션, 민감한 API | 높음 |

---

### 🔧 개별 권한 할당 방법

#### Step 1: User Assignment Required 설정

1. **Azure Portal** > **Entra ID** > **Enterprise applications**
2. **MCP-API** 앱 선택
3. **Properties** 메뉴 클릭
4. **Assignment required?** 를 **Yes** 로 설정
5. **Save** 클릭

```
Assignment required? = Yes
→ 명시적으로 할당된 사용자/그룹만 API에 접근 가능
→ 할당되지 않은 사용자는 토큰 발급 거부됨
```

---

#### Step 2: 사용자 또는 그룹 할당

1. **Enterprise applications** > **MCP-API** > **Users and groups**
2. **Add user/group** 클릭
3. 할당할 사용자 또는 그룹 선택
4. (선택) 역할 선택 (앱에 역할이 정의된 경우)
5. **Assign** 클릭

---

#### Step 3: Client 앱에도 동일하게 적용

MCP-Client 앱에 대해서도 위 과정을 반복합니다:

1. **Enterprise applications** > **MCP-Client** 선택
2. **Properties** > **Assignment required?** = **Yes**
3. **Users and groups** > 필요한 사용자/그룹 할당

---

### 💻 Azure CLI로 설정하기

```bash
# 1. Enterprise App의 Object ID 조회
API_SP_OBJECT_ID=$(az ad sp list --display-name "MCP-API-{suffix}" --query "[0].id" -o tsv)

# 2. Assignment Required 설정 (appRoleAssignmentRequired = true)
az rest --method PATCH \
  --url "https://graph.microsoft.com/v1.0/servicePrincipals/$API_SP_OBJECT_ID" \
  --body '{"appRoleAssignmentRequired": true}'

# 3. 특정 사용자에게 앱 역할 할당
USER_OBJECT_ID="사용자의-object-id"
az rest --method POST \
  --url "https://graph.microsoft.com/v1.0/servicePrincipals/$API_SP_OBJECT_ID/appRoleAssignments" \
  --body "{
    \"principalId\": \"$USER_OBJECT_ID\",
    \"resourceId\": \"$API_SP_OBJECT_ID\",
    \"appRoleId\": \"00000000-0000-0000-0000-000000000000\"
  }"
```

> 💡 **appRoleId**: `00000000-0000-0000-0000-000000000000`은 기본 역할(Default Access)을 의미합니다.
> 커스텀 역할을 정의한 경우 해당 역할의 ID를 사용하세요.

---

### 🔐 조건부 액세스 정책 (고급)

더 세밀한 제어가 필요한 경우 **조건부 액세스 정책**을 설정할 수 있습니다:

| 조건 | 예시 |
|------|------|
| **위치** | 특정 IP 범위에서만 접근 허용 |
| **디바이스** | Compliant 디바이스에서만 접근 |
| **MFA** | 다단계 인증 필수 |
| **시간** | 업무 시간에만 접근 허용 |
| **위험 수준** | 로그인 위험이 낮을 때만 허용 |

**설정 경로**: Entra ID > **Security** > **Conditional Access** > **New policy**

---

### ⚠️ 주의사항

1. **Client Credentials Flow에서의 제한**
   - Client Credentials Flow는 사용자 컨텍스트가 없으므로 사용자 할당이 적용되지 않습니다
   - 앱 자체에 대한 접근 제어는 **앱 역할(App Roles)**로 구현해야 합니다

2. **Authorization Code Flow에서는 완전히 적용**
   - 사용자 로그인이 포함된 흐름에서는 User Assignment가 완전히 작동합니다
   - 할당되지 않은 사용자는 로그인 시 접근 거부됨

3. **그룹 할당 권장**
   - 개별 사용자보다 **그룹**에 할당하는 것이 관리가 용이합니다
   - Entra ID 그룹 또는 Microsoft 365 그룹 사용 가능

## 4.2 JWT 검증 정책 적용

APIM의 `validate-jwt` 정책을 사용하여 들어오는 요청의 JWT 토큰을 검증합니다.

---

### 🔍 검증 항목

| 항목 | JWT 클레임 | 검증 내용 |
|------|-----------|----------|
| **발급자** | `iss` | Entra ID 발급 토큰인지 확인 |
| **대상** | `aud` | 우리 API용 토큰인지 확인 |
| **만료** | `exp` | 토큰이 만료되지 않았는지 확인 |
| **서명** | - | JWKS로 토큰 위변조 여부 확인 |

---

### 📄 validate-jwt 정책 구조

```xml
<validate-jwt header-name="Authorization" 
              failed-validation-httpcode="401"
              failed-validation-error-message="Unauthorized">
    
    <!-- OpenID Configuration 자동 검색 -->
    <openid-config url="https://login.microsoftonline.com/{tenant}/v2.0/.well-known/openid-configuration" />
    
    <!-- 토큰 수신자 (Audience) 검증 -->
    <audiences>
        <audience>api://{api-app-id}</audience>
    </audiences>
    
    <!-- 토큰 발급자 (Issuer) 검증 -->
    <issuers>
        <issuer>https://sts.windows.net/{tenant-id}/</issuer>
        <issuer>https://login.microsoftonline.com/{tenant-id}/v2.0</issuer>
    </issuers>
</validate-jwt>
```

---

### 🔑 주요 속성 설명

| 속성 | 설명 |
|------|------|
| `header-name` | JWT를 찾을 HTTP 헤더 (`Authorization`) |
| `failed-validation-httpcode` | 검증 실패 시 반환할 HTTP 상태 코드 |
| `require-expiration-time` | `exp` 클레임 필수 여부 |
| `require-signed-tokens` | 서명된 토큰만 허용 |

---

### ⚠️ v1.0 vs v2.0 엔드포인트

Entra ID는 두 가지 버전의 토큰을 발급할 수 있습니다:

| 버전 | Issuer 형식 | 특징 |
|------|------------|------|
| **v1.0** | `https://sts.windows.net/{tenant}/` | 레거시, 일부 앱에서 사용 |
| **v2.0** | `https://login.microsoftonline.com/{tenant}/v2.0` | 최신 표준, 권장 |

> 💡 **권장**: 두 issuer 모두 허용하여 호환성 확보

In [22]:
# JWT 검증 정책 APIM에 직접 적용
import subprocess
import json
import requests

print("🔐 JWT 검증 정책 적용")
print("=" * 80)

# 환경 변수에서 값 가져오기
tenant_id = WORKSHOP_CONFIG.get("tenant_id", "")
api_app_id = os.getenv("ENTRA_API_APP_ID", "")
subscription_id = WORKSHOP_CONFIG.get("subscription_id", "")
apim_name = WORKSHOP_CONFIG.get("apim_name", "")
rg_name = WORKSHOP_CONFIG.get("resource_group", "rg-mcp-lab")

if not all([tenant_id, api_app_id, subscription_id, apim_name]):
    print("❌ 필요한 환경 변수가 설정되지 않았습니다.")
    print("   먼저 4.1 Entra ID 앱 등록 셀을 실행하세요.")
else:
    # JWT 검증 정책 생성
    jwt_policy = f'''<policies>
    <inbound>
        <base />
        <set-backend-service backend-id="mcp-functions-backend" />
        <validate-jwt header-name="Authorization" 
                      failed-validation-httpcode="401" 
                      failed-validation-error-message="Unauthorized: Invalid or missing JWT token"
                      require-expiration-time="true"
                      require-signed-tokens="true">
            <openid-config url="https://login.microsoftonline.com/{tenant_id}/v2.0/.well-known/openid-configuration" />
            <audiences>
                <audience>api://{api_app_id}</audience>
            </audiences>
            <issuers>
                <issuer>https://sts.windows.net/{tenant_id}/</issuer>
                <issuer>https://login.microsoftonline.com/{tenant_id}/v2.0</issuer>
            </issuers>
        </validate-jwt>
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <set-header name="X-MCP-Protocol-Version" exists-action="override">
            <value>2024-11-05</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>'''

    print("📋 적용할 JWT 검증 정책:")
    print("-" * 40)
    print(jwt_policy)
    print("-" * 40)
    print()
    
    # Azure 액세스 토큰 획득
    print("1️⃣ Azure 액세스 토큰 획득 중...")
    token_result = subprocess.run([
        'az', 'account', 'get-access-token',
        '--resource', 'https://management.azure.com/',
        '--query', 'accessToken',
        '--output', 'tsv'
    ], capture_output=True, text=True)
    
    if token_result.returncode != 0:
        print(f"   ❌ 토큰 획득 실패: {token_result.stderr}")
    else:
        access_token = token_result.stdout.strip()
        print("   ✅ 토큰 획득 완료")
        
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }
        
        # APIM에 JWT 정책 적용
        print()
        print("2️⃣ APIM MCP API에 JWT 정책 적용 중...")
        
        policy_url = (
            f"https://management.azure.com/subscriptions/{subscription_id}"
            f"/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement"
            f"/service/{apim_name}/apis/mcp-api/policies/policy?api-version=2024-05-01"
        )
        
        policy_body = {
            "properties": {
                "format": "xml",
                "value": jwt_policy
            }
        }
        
        response = requests.put(policy_url, headers=headers, json=policy_body)
        
        if response.status_code in [200, 201]:
            print("   ✅ JWT 검증 정책 적용 완료!")
            print()
            print("=" * 80)
            print("📌 정책 요약:")
            print(f"   ✅ OpenID Config: https://login.microsoftonline.com/{tenant_id}/v2.0/.well-known/openid-configuration")
            print(f"   ✅ Audience: api://{api_app_id}")
            print(f"   ✅ Issuer (v1): https://sts.windows.net/{tenant_id}/")
            print(f"   ✅ Issuer (v2): https://login.microsoftonline.com/{tenant_id}/v2.0")
            print()
            print("💡 이제 MCP API 호출 시 유효한 JWT 토큰이 필요합니다.")
            print("   Authorization: Bearer <access_token>")
        else:
            print(f"   ❌ 정책 적용 실패: {response.status_code}")
            print(f"   {response.text[:300]}")

🔐 JWT 검증 정책 적용
📋 적용할 JWT 검증 정책:
----------------------------------------
<policies>
    <inbound>
        <base />
        <set-backend-service backend-id="mcp-functions-backend" />
        <validate-jwt header-name="Authorization" 
                      failed-validation-httpcode="401" 
                      failed-validation-error-message="Unauthorized: Invalid or missing JWT token"
                      require-expiration-time="true"
                      require-signed-tokens="true">
            <openid-config url="https://login.microsoftonline.com/38bd4dc5-c85c-4efd-a73a-3dfe7d6f632e/v2.0/.well-known/openid-configuration" />
            <audiences>
                <audience>api://56a56150-850b-4889-8bab-cc775012d4ba</audience>
            </audiences>
            <issuers>
                <issuer>https://sts.windows.net/38bd4dc5-c85c-4efd-a73a-3dfe7d6f632e/</issuer>
                <issuer>https://login.microsoftonline.com/38bd4dc5-c85c-4efd-a73a-3dfe7d6f632e/v2.0</issuer>
       

## 4.3 토큰 발급 및 테스트

OAuth 2.0 Client Credentials Flow로 토큰을 발급받고 APIM을 통해 MCP 서버를 호출합니다.

---

### 📊 토큰 요청 흐름

```
┌──────────────┐                              ┌─────────────────┐
│ MCP-Client   │── POST /oauth2/v2.0/token ──▶│   Entra ID      │
│              │   ├─ client_id               │                 │
│              │   ├─ client_secret           │                 │
│              │   ├─ scope                   │                 │
│              │   └─ grant_type              │                 │
│              │◀────── access_token ─────────│                 │
└──────────────┘                              └─────────────────┘
```

---

### 📝 토큰 요청 파라미터

| 파라미터 | 값 | 설명 |
|----------|-----|------|
| `client_id` | MCP-Client App ID | 토큰 요청하는 앱 |
| `client_secret` | 앱 비밀 키 | 앱 인증용 |
| `scope` | `api://{id}/.default` | 요청할 권한 범위 |
| `grant_type` | `client_credentials` | 인증 흐름 유형 |

> ⚠️ **중요**: Client Credentials Flow에서는 scope에 `.default` 접미사를 붙여야 합니다.
> 이는 "앱에 할당된 모든 권한을 요청"한다는 의미입니다.

---

### 🎫 발급된 JWT 토큰 구조

```json
{
  "aud": "api://12345678-...",      // 토큰 수신자 (API App ID)
  "iss": "https://sts.windows.net/87654321-.../",  // 발급자
  "iat": 1702200000,                // 발급 시간
  "exp": 1702203600,                // 만료 시간 (1시간 후)
  "azp": "aaaaaaaa-...",            // 토큰 요청한 앱 (Client App ID)
  "oid": "00000000-...",            // Object ID
  "sub": "00000000-...",            // Subject
  "tid": "87654321-...",            // Tenant ID
  "ver": "2.0"                      // 토큰 버전
}
```

---

### 🧪 테스트 순서

아래 코드는 다음 단계를 자동으로 수행합니다:

1. **APIM에 JWT 정책 적용** - `validate-jwt` 정책을 MCP API에 추가
2. **토큰 발급** - Entra ID에서 Access Token 획득
3. **APIM Subscription Key 확인** - API 호출에 필요한 키 조회
4. **JWT 인증 테스트** - Bearer 토큰과 함께 API 호출

---

### 🔧 수동 테스트 (cURL)

```bash
# 1. 토큰 발급
TOKEN=$(curl -s -X POST \
  "https://login.microsoftonline.com/{tenant-id}/oauth2/v2.0/token" \
  -d "client_id={client-app-id}" \
  -d "client_secret={client-secret}" \
  -d "scope=api://{api-app-id}/.default" \
  -d "grant_type=client_credentials" \
  | jq -r '.access_token')

# 2. APIM 호출 (JWT 인증)
curl -X GET "https://{apim-name}.azure-api.net/mcp/health" \
  -H "Authorization: Bearer $TOKEN" \
  -H "Ocp-Apim-Subscription-Key: {subscription-key}"
```

---

### ✅ 예상 결과

**인증 성공 (200 OK):**
```json
{
  "status": "healthy",
  "server": "Azure-MCP-Functions-Server",
  "version": "2024-11-05"
}
```

**인증 실패 (401 Unauthorized):**
```json
{
  "statusCode": 401,
  "message": "Unauthorized: Invalid or missing JWT token"
}
```

In [23]:
# 토큰 발급 및 JWT 정책 적용
import subprocess
import json

tenant_id = WORKSHOP_CONFIG.get("tenant_id", "")
client_id = os.getenv("ENTRA_CLIENT_APP_ID", "")
client_secret = os.getenv("ENTRA_CLIENT_SECRET", "")
api_app_id = os.getenv("ENTRA_API_APP_ID", "")
subscription_id = WORKSHOP_CONFIG.get("subscription_id", "")
apim_name = WORKSHOP_CONFIG.get("apim_name", "")
rg_name = WORKSHOP_CONFIG.get("resource_group", "rg-mcp-lab")

if all([tenant_id, client_id, client_secret, api_app_id]):
    
    # Step 1: APIM에 JWT 검증 정책 적용
    print("1️⃣ APIM에 JWT 검증 정책 적용 중...")
    
    policy_url = (
        f"https://management.azure.com/subscriptions/{subscription_id}"
        f"/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement"
        f"/service/{apim_name}/apis/mcp-api/policies/policy?api-version=2024-05-01"
    )
    
    # Client Credentials 흐름을 위한 JWT 정책 (scp 대신 aud만 검증)
    jwt_api_policy = f'''<policies>
    <inbound>
        <base />
        <set-backend-service backend-id="mcp-functions-backend" />
        <validate-jwt header-name="Authorization" 
                      failed-validation-httpcode="401" 
                      failed-validation-error-message="Unauthorized: Invalid or missing JWT token"
                      require-expiration-time="true"
                      require-signed-tokens="true">
            <openid-config url="https://login.microsoftonline.com/{tenant_id}/v2.0/.well-known/openid-configuration" />
            <audiences>
                <audience>api://{api_app_id}</audience>
            </audiences>
            <issuers>
                <issuer>https://sts.windows.net/{tenant_id}/</issuer>
                <issuer>https://login.microsoftonline.com/{tenant_id}/v2.0</issuer>
            </issuers>
        </validate-jwt>
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <set-header name="X-MCP-Protocol-Version" exists-action="override">
            <value>2024-11-05</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>'''
    
    policy_body = json.dumps({
        "properties": {
            "format": "xml",
            "value": jwt_api_policy
        }
    })
    
    policy_result = subprocess.run([
        'az', 'rest', '--method', 'put',
        '--url', policy_url,
        '--body', policy_body,
        '-o', 'none'
    ], capture_output=True, text=True)
    
    if policy_result.returncode == 0:
        print("   ✅ JWT 검증 정책 적용 완료!")
    else:
        print(f"   ❌ 정책 적용 실패: {policy_result.stderr[:200]}")
    
    # Step 2: 토큰 발급
    print()
    print("2️⃣ OAuth 2.0 토큰 발급 중...")
    
    token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    token_data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": f"api://{api_app_id}/.default",
        "grant_type": "client_credentials"
    }
    
    try:
        response = requests.post(token_url, data=token_data, timeout=10)
        
        if response.status_code != 200:
            print(f"❌ 토큰 발급 실패: {response.status_code}")
            error_data = response.json()
            print(f"   Error: {error_data.get('error', 'N/A')}")
            print(f"   Description: {error_data.get('error_description', 'N/A')}")
        else:
            token_response = response.json()
            access_token = token_response.get("access_token", "")
        
            if access_token:
                print("   ✅ 토큰 발급 성공!")
                print(f"   Token Type: {token_response.get('token_type', 'N/A')}")
                print(f"   Expires In: {token_response.get('expires_in', 'N/A')} seconds")
                
                # 토큰을 환경 변수에 저장
                os.environ["MCP_ACCESS_TOKEN"] = access_token
                WORKSHOP_CONFIG['mcp_access_token'] = access_token
                
                # Step 3: APIM Subscription Key 가져오기
                print()
                print("3️⃣ APIM Subscription Key 확인...")
                subscription_key = WORKSHOP_CONFIG.get('apim_subscription_key', '')
                
                if not subscription_key:
                    list_secrets_url = (
                        f"https://management.azure.com/subscriptions/{subscription_id}"
                        f"/resourceGroups/{rg_name}/providers/Microsoft.ApiManagement"
                        f"/service/{apim_name}/subscriptions/master/listSecrets?api-version=2024-05-01"
                    )
                    
                    key_result = subprocess.run([
                        'az', 'rest', '--method', 'post',
                        '--url', list_secrets_url,
                        '--query', 'primaryKey',
                        '-o', 'tsv'
                    ], capture_output=True, text=True)
                    
                    if key_result.returncode == 0:
                        subscription_key = key_result.stdout.strip()
                        WORKSHOP_CONFIG['apim_subscription_key'] = subscription_key
                
                print(f"   ✅ Subscription Key: {subscription_key[:10]}...")
                
                # Step 4: 정책 적용 대기 후 테스트
                import time
                print()
                print("⏳ 정책 적용 대기 중... (5초)")
                time.sleep(5)
                
                # Step 5: JWT 인증 테스트
                print()
                print("4️⃣ JWT 인증 테스트")
                apim_url = WORKSHOP_CONFIG.get("apim_gateway_url", "")
                if apim_url:
                    print(f"   🔍 {apim_url}/mcp/health")
                    headers = {
                        "Authorization": f"Bearer {access_token}",
                        "Ocp-Apim-Subscription-Key": subscription_key
                    }
                    
                    test_response = requests.get(f"{apim_url}/mcp/health", headers=headers, timeout=15)
                    print(f"   상태: {test_response.status_code}")
                    
                    if test_response.status_code == 200:
                        print(f"   ✅ JWT 인증 성공!")
                        print(f"   응답: {test_response.json()}")
                        print()
                        print("=" * 60)
                        print("✅ Entra ID JWT 인증 설정 완료!")
                        print("=" * 60)
                    elif test_response.status_code == 401:
                        print("   ❌ 인증 실패 (401 Unauthorized)")
                        print(f"   응답: {test_response.text[:300]}")
                    else:
                        print(f"   ❌ 오류: {test_response.text[:300]}")
            else:
                print("❌ 토큰을 받지 못했습니다.")
            
    except requests.exceptions.RequestException as e:
        print(f"❌ 토큰 발급 실패: {e}")
        
else:
    print("⚠️ Entra ID 설정이 완료되지 않았습니다.")
    print("   먼저 셀 38을 실행하여 Entra ID 앱을 등록하세요.")

1️⃣ APIM에 JWT 검증 정책 적용 중...
   ✅ JWT 검증 정책 적용 완료!

2️⃣ OAuth 2.0 토큰 발급 중...
   ✅ 토큰 발급 성공!
   Token Type: Bearer
   Expires In: 3599 seconds

3️⃣ APIM Subscription Key 확인...
   ✅ Subscription Key: 90c98e4841...

⏳ 정책 적용 대기 중... (5초)

4️⃣ JWT 인증 테스트
   🔍 https://mcp-dev-apim-i6ht.azure-api.net/mcp/health
   상태: 200
   ✅ JWT 인증 성공!
   응답: {'status': 'healthy', 'timestamp': '2025-12-11T01:48:32.822529', 'server': 'Azure-MCP-Functions-Server', 'version': '1.0.0'}

✅ Entra ID JWT 인증 설정 완료!


<a id='5'></a>
# 5️⃣ External Entra ID 연계 (고급)

외부 Entra ID 테넌트 사용자(B2B Guest)가 MCP 서버에 접근하는 시나리오를 구현합니다.

## 학습 목표
- Cross-tenant 접근 설정
- B2B Guest 사용자 초대
- External 사용자 토큰 검증

## 5.1 B2B Guest 사용자 초대

외부 테넌트의 사용자를 Guest로 초대하여 MCP 서버에 접근할 수 있도록 합니다.

### 시나리오 개요
- **조직 A (contoso.com)**: APIM + MCP 서버 호스팅
- **조직 B (fabrikam.com)**: 파트너 조직
- **목표**: fabrikam.com 사용자가 contoso.com의 MCP 서버 접근

### Step 1: Cross-tenant Access 설정 (조직 A)
1. Azure Portal > **Entra ID** > **External Identities**
2. **Cross-tenant access settings** 클릭
3. **Organizational settings** > **Add organization**
4. Tenant ID 또는 Domain 입력 (fabrikam.com)
5. Inbound access 설정:
   - B2B collaboration: ✅ **Allow**
   - Applications: **MCP-API** 선택

### Step 2: Guest 사용자 초대
1. Entra ID > **Users** > **New user** > **Invite external user**
2. Email: `user@fabrikam.com`
3. Display name: `Fabrikam User`
4. **Invite** 클릭 → 초대 이메일 발송됨

### Step 3: 앱 권한 부여
1. Entra ID > **Enterprise applications** > **MCP-API**
2. **Users and groups** > **Add user/group**
3. Guest 사용자 선택 (`user_fabrikam.com#EXT#`)
4. **Assign** 클릭

### Step 4: JWT 정책 수정 (APIM)

`validate-jwt` 정책의 `<issuers>` 태그에 외부 테넌트 추가:

```xml
<issuer>https://sts.windows.net/{fabrikam-tenant-id}/</issuer>
```

또는 `tid` (Tenant ID) 클레임으로 검증:

```xml
<claim name="tid" match="any">
    <value>{your-tenant-id}</value>
    <value>{external-tenant-id}</value>
</claim>
```

### Step 5: External 사용자 토큰 테스트
1. External 사용자로 로그인
2. 토큰 발급 (동일한 Client Credentials Flow)
3. APIM 호출
4. JWT에서 `tid` 클레임 확인 (외부 테넌트 ID)

### 💡 보안 고려사항
- MFA 필수 설정 (조건부 액세스)
- IP 제한
- 세션 타임아웃
- 감사 로그 모니터링

## 5.2 Custom Identity Provider (OAuth2/OpenID/SAML) 연동

Entra ID를 사용하지 않는 외부 조직이나 사용자를 위해 **Custom Identity Provider**를 APIM에 연동하는 방법입니다.

### 지원 IdP 종류
| 프로토콜 | IdP 예시 |
|---------|---------|
| **OpenID Connect (OIDC)** | Google, GitHub, Okta, Auth0, Ping Identity, Keycloak |
| **OAuth 2.0** | 대부분의 소셜 로그인 제공자 |
| **SAML 2.0** | 기업용 SSO (Okta, OneLogin, ADFS) |

### 연동 아키텍처
```
┌─────────────────┐     ┌─────────────────┐     ┌──────────────────┐
│  MCP Client     │────▶│  Custom IdP     │────▶│  APIM Gateway    │
│  (External App) │ 1.  │  (Okta/Auth0)   │ 2.  │  JWT Validation  │
└─────────────────┘     └─────────────────┘     └────────┬─────────┘
                                                         │ 3.
                                                         ▼
                                                ┌──────────────────┐
                                                │  Azure Functions │
                                                │  MCP Server      │
                                                └──────────────────┘
```
1. 클라이언트가 Custom IdP에서 토큰 발급
2. 발급받은 JWT를 APIM에 전달
3. APIM에서 JWT 검증 후 Backend 호출

---

### 5.2.1 OpenID Connect (OIDC) 연동 - 가장 권장되는 방식

**장점**: 표준화된 프로토콜, 자동 Discovery, 간편한 설정

#### Google IdP 연동

1. [Google Cloud Console](https://console.cloud.google.com) → **APIs & Services** → **Credentials**
2. OAuth 2.0 Client ID 생성
3. Authorized redirect URIs에 MCP 클라이언트 URL 추가

**APIM Inbound Policy:**
```xml
<validate-jwt header-name="Authorization" failed-validation-httpcode="401">
    <openid-config url="https://accounts.google.com/.well-known/openid-configuration" />
    <required-claims>
        <claim name="aud" match="any">
            <value>YOUR_GOOGLE_CLIENT_ID.apps.googleusercontent.com</value>
        </claim>
    </required-claims>
</validate-jwt>
```

#### Auth0 IdP 연동

1. [Auth0 Dashboard](https://manage.auth0.com) → **Applications** → **APIs** → **Create API**
2. Identifier(audience) 설정 (예: `https://api.example.com`)
3. Machine-to-Machine 또는 SPA 애플리케이션 생성

**APIM Inbound Policy:**
```xml
<validate-jwt header-name="Authorization" failed-validation-httpcode="401">
    <openid-config url="https://YOUR_TENANT.auth0.com/.well-known/openid-configuration" />
    <required-claims>
        <claim name="aud" match="any">
            <value>https://api.example.com</value>
        </claim>
    </required-claims>
</validate-jwt>
```

#### Okta IdP 연동

1. [Okta Admin](https://YOUR_ORG.okta.com/admin) → **Security** → **API**
2. Authorization Server 선택 또는 생성
3. Access Policies 및 Scopes 설정

**APIM Inbound Policy:**
```xml
<validate-jwt header-name="Authorization" failed-validation-httpcode="401">
    <openid-config url="https://YOUR_ORG.okta.com/.well-known/openid-configuration" />
    <required-claims>
        <claim name="aud" match="any">
            <value>api://your-api-audience</value>
        </claim>
    </required-claims>
</validate-jwt>
```

---

### 5.2.2 다중 IdP 지원 설정

여러 IdP를 동시에 지원해야 하는 경우, `<choose>` 문을 사용하여 토큰 발급자별로 분기 처리합니다.

```xml
<inbound>
    <choose>
        <!-- Google 토큰 -->
        <when condition="@(context.Request.Headers.GetValueOrDefault("Authorization","")
                         .Contains("accounts.google.com"))">
            <validate-jwt header-name="Authorization">
                <openid-config url="https://accounts.google.com/.well-known/openid-configuration" />
            </validate-jwt>
        </when>
        <!-- Auth0 토큰 -->
        <when condition="@(context.Request.Headers.GetValueOrDefault("Authorization","")
                         .Contains("auth0.com"))">
            <validate-jwt header-name="Authorization">
                <openid-config url="https://YOUR_TENANT.auth0.com/.well-known/openid-configuration" />
            </validate-jwt>
        </when>
        <!-- Entra ID 토큰 (내부 사용자) -->
        <when condition="@(context.Request.Headers.GetValueOrDefault("Authorization","")
                         .Contains("login.microsoftonline.com"))">
            <validate-jwt header-name="Authorization">
                <openid-config url="https://login.microsoftonline.com/{tenant}/v2.0/.well-known/openid-configuration" />
            </validate-jwt>
        </when>
        <otherwise>
            <return-response>
                <set-status code="401" reason="Unauthorized" />
                <set-body>Invalid or unsupported token issuer</set-body>
            </return-response>
        </otherwise>
    </choose>
</inbound>
```

> 💡 **팁**: 실제 프로덕션에서는 JWT의 `iss` claim을 파싱하여 조건을 분기하는 것이 더 정확합니다.

---

### 5.2.3 SAML 2.0 연동

SAML은 브라우저 기반 SSO에 적합하며, API 인증에는 추가 변환이 필요합니다.

#### SAML → JWT 변환 아키텍처
```
┌─────────────┐     ┌───────────────┐     ┌─────────────┐     ┌──────────────┐
│  MCP Client │────▶│  SAML IdP     │────▶│  Token STS  │────▶│  APIM        │
│             │     │  (ADFS/Okta)  │     │  (Azure)    │     │  validate-jwt│
└─────────────┘     └───────────────┘     └─────────────┘     └──────────────┘
      1. SAML 로그인           2. SAML Assertion        3. JWT로 API 호출
```

#### 방법 1: Azure AD Federation 사용
1. Entra ID → **Enterprise Applications** → **New Application**
2. SAML SSO 설정 후 Federation 구성
3. SAML 사용자가 Entra ID를 통해 JWT 발급 가능

#### 방법 2: Custom Token Service (STS)
Azure Function으로 SAML Assertion을 JWT로 변환하는 서비스 구현

---

### IdP 연동 방식 비교

| 방식 | 장점 | 단점 | 권장 시나리오 |
|------|------|------|---------------|
| **OIDC** | 표준화, 자동 Discovery, APIM 직접 지원 | - | ✅ API 인증 (최우선 권장) |
| **OAuth 2.0** | 널리 지원됨, 유연한 스코프 | Discovery 없음 | SPA/모바일 앱 |
| **SAML 2.0** | 기업 SSO 표준, 보안성 높음 | API 직접 사용 불가, 변환 필요 | 웹 SSO 연동 |

## 5.3 Azure Entra ID B2C를 통한 외부 사용자 인증

**Azure Entra ID B2C**는 **소비자/외부 사용자(Consumer)**를 위한 ID 관리 서비스입니다. 소셜 계정 로그인, 이메일 가입 등을 지원하며, MCP API에 대한 외부 사용자 인증에 적합합니다.

### B2B vs B2C 비교

| 구분 | B2B (Guest Users) | B2C (Consumer Identity) |
|------|-------------------|-------------------------|
| **대상 사용자** | 파트너 조직의 비즈니스 사용자 | 일반 소비자, 외부 개인 사용자 |
| **인증 방식** | 파트너의 Entra ID/IdP | 소셜 로그인, 이메일 가입 |
| **계정 관리** | 파트너 조직이 관리 | B2C 테넌트에서 관리 |
| **브랜딩** | 제한적 커스터마이징 | 완전한 UI 커스터마이징 |
| **비용** | MAU 5만 명까지 무료 | MAU 5만 명까지 무료 |
| **사용 시나리오** | 기업 간 협업 | 고객용 앱/API |

### 연동 아키텍처
```
┌──────────────────┐     ┌─────────────────────┐     ┌──────────────────┐
│  MCP Client      │────▶│  Azure AD B2C       │────▶│  APIM Gateway    │
│  (External User) │     │  - Google/Facebook  │     │  JWT Validation  │
│                  │ 1.  │  - Email Sign-up    │ 2.  │  (B2C 발급 토큰) │
└──────────────────┘     └─────────────────────┘     └────────┬─────────┘
                                                              │ 3.
                                                              ▼
                                                     ┌──────────────────┐
                                                     │  Azure Functions │
                                                     │  MCP Server      │
                                                     └──────────────────┘
```

---

### 5.3.1 B2C 테넌트 설정

#### Step 1: B2C 테넌트 생성
1. Azure Portal → **"Azure AD B2C"** 검색 → **Create**
2. 새 테넌트 이름 설정 (예: `mcpworkshopb2c`)
3. 도메인: `mcpworkshopb2c.onmicrosoft.com`

#### Step 2: 애플리케이션 등록
1. B2C 테넌트 → **App registrations** → **New registration**
2. 이름: `MCP API Client`
3. Redirect URI: 클라이언트 앱 URL (SPA의 경우 `http://localhost:3000`)
4. Implicit grant: **ID tokens**, **Access tokens** 체크
5. API permissions: 필요한 경우 추가

#### Step 3: User Flow 생성
1. **User flows** → **New user flow** → **Sign up and sign in (Recommended)**
2. 이름: `B2C_1_signupsignin`
3. Identity providers: Email signup, Google, Facebook 등 선택
4. User attributes: Display name, Email 등 수집할 정보 선택
5. Application claims: 토큰에 포함할 클레임 선택

#### Step 4: 소셜 IdP 추가 (선택사항)

**Google 추가:**
1. Identity providers → **Google**
2. Client ID/Secret: Google Cloud Console에서 생성
3. Scope: `openid`, `email`, `profile`

**GitHub 추가:**
1. Identity providers → **GitHub**
2. Client ID/Secret: GitHub Developer Settings에서 OAuth App 생성

---

### 5.3.2 APIM에서 B2C 토큰 검증

B2C 테넌트가 설정되면 APIM에서 B2C 발급 토큰을 검증할 수 있습니다.

```xml
<validate-jwt header-name="Authorization" failed-validation-httpcode="401">
    <!-- B2C 테넌트의 OpenID Configuration URL -->
    <openid-config 
        url="https://YOUR_B2C_TENANT.b2clogin.com/YOUR_B2C_TENANT.onmicrosoft.com/B2C_1_signupsignin/v2.0/.well-known/openid-configuration" />
    <audiences>
        <!-- B2C에 등록된 API 클라이언트 ID -->
        <audience>YOUR_B2C_CLIENT_ID</audience>
    </audiences>
    <required-claims>
        <claim name="scp" match="any">
            <value>read</value>
            <value>write</value>
        </claim>
    </required-claims>
</validate-jwt>
```

B2C 토큰에서 사용자 정보 추출하여 백엔드로 전달:

```xml
<set-header name="X-User-Email" exists-action="override">
    <value>@(context.Request.Headers.GetValueOrDefault("Authorization","")
            .AsJwt()?.Claims.GetValueOrDefault("emails", ""))</value>
</set-header>
<set-header name="X-User-Name" exists-action="override">
    <value>@(context.Request.Headers.GetValueOrDefault("Authorization","")
            .AsJwt()?.Claims.GetValueOrDefault("name", ""))</value>
</set-header>
```

> ⚠️ **주의**: B2C의 OpenID Configuration URL은 User Flow 이름(`B2C_1_signupsignin`)을 포함해야 합니다.

---

### 5.3.3 B2C 토큰 획득 예시

#### JavaScript (SPA) - MSAL.js
```javascript
const msalConfig = {
    auth: {
        clientId: "YOUR_B2C_CLIENT_ID",
        authority: "https://YOUR_TENANT.b2clogin.com/YOUR_TENANT.onmicrosoft.com/B2C_1_signupsignin",
        knownAuthorities: ["YOUR_TENANT.b2clogin.com"],
        redirectUri: "http://localhost:3000"
    }
};

const msalInstance = new msal.PublicClientApplication(msalConfig);

// 로그인
const response = await msalInstance.loginPopup({
    scopes: ["openid", "profile", "YOUR_B2C_CLIENT_ID"]
});
const accessToken = response.accessToken;

// MCP API 호출
const apiResponse = await fetch("https://YOUR_APIM.azure-api.net/mcp/tools/list", {
    method: "POST",
    headers: {
        "Authorization": `Bearer ${accessToken}`,
        "Content-Type": "application/json"
    },
    body: JSON.stringify({
        jsonrpc: "2.0", id: 1, method: "tools/list", params: {}
    })
});
```

#### Python (CLI) - MSAL
```python
import msal

app = msal.PublicClientApplication(
    "YOUR_B2C_CLIENT_ID",
    authority="https://YOUR_TENANT.b2clogin.com/YOUR_TENANT.onmicrosoft.com/B2C_1_signupsignin"
)

# Device Code Flow (CLI 앱용)
flow = app.initiate_device_flow(scopes=["openid", "profile"])
print(flow["message"])  # 사용자에게 인증 URL 표시

result = app.acquire_token_by_device_flow(flow)
access_token = result["access_token"]
```

---

### 5.3.4 B2C 사용 시 고려사항

#### ✅ 장점
- **완전한 UI 커스터마이징**: 브랜드에 맞는 로그인 화면 구성
- **다양한 인증 옵션**: 이메일, SMS, 소셜 로그인 모두 지원
- **셀프 서비스**: 사용자가 직접 가입/비밀번호 재설정 가능
- **확장성**: MAU 수백만까지 자동 확장
- **규정 준수**: GDPR, SOC2 등 인증 지원

#### ⚠️ 제한사항
- **별도 테넌트 필요**: 기존 Entra ID 테넌트와 분리된 B2C 테넌트 생성 필요
- **설정 복잡도**: User Flow/Custom Policy 설정에 학습 필요
- **비용**: 5만 MAU 이후 유료 (P1: $0.00325/MAU, P2: $0.01625/MAU)

---

### Section 5 요약 - 인증 방식별 권장 시나리오

| 인증 방식 | 대상 사용자 | 설정 복잡도 | 권장 시나리오 |
|-----------|-------------|-------------|---------------|
| **5.1 B2B Guest** | 파트너 조직의 Entra ID 사용자 | 낮음 | 기업 간 협업 |
| **5.2 Custom IdP** | 외부 조직의 자체 IdP 사용자 | 중간 | 기업용 OIDC/OAuth/SAML 연동 |
| **5.3 B2C** | 일반 소비자, 개인 사용자 | 중간~높음 | 고객용 앱, 소셜 로그인 |
| **5.4 API 키** | 서버/개발자 | 🟢 매우 낮음 | 개발/테스트, 서버 간 통신 |

> 💡 **팁**: 여러 시나리오를 동시에 지원해야 하는 경우, Section 5.2.2의 **다중 IdP 지원** 패턴을 활용하여 Entra ID, B2C, Custom IdP 토큰을 모두 검증할 수 있습니다. 개발 환경에서는 5.4 API 키로 빠르게 시작하고, 프로덕션에서는 JWT 기반 인증으로 전환하는 것이 권장됩니다.

## 5.4 API Key 인증 (Subscription Key)

APIM은 JWT 인증 외에도 **Subscription Key (API Key)** 기반 인증을 제공합니다. 이 방식은 설정이 간단하고 Rate Limiting과 함께 사용하기 좋습니다.

> ⚠️ **참고**: 이 섹션은 개념 설명용입니다. 실제 구현은 Azure Portal 또는 Azure CLI에서 수행합니다.

---

### 5.4.1 Subscription Key 생성

APIM에서 Subscription을 생성하여 API Key를 발급받습니다.

**Azure Portal에서:**
1. APIM 리소스 → **Subscriptions** 메뉴
2. **+ Add subscription** 클릭
3. 이름, 스코프(API 또는 Product) 선택
4. Primary/Secondary Key 자동 생성

**Azure CLI로:**
```bash
# Subscription 목록 조회
az apim subscription list \
    --resource-group <resource-group> \
    --service-name <apim-name> \
    --output table

# 새 Subscription 생성
az apim subscription create \
    --resource-group <resource-group> \
    --service-name <apim-name> \
    --subscription-id "partner-app-001" \
    --display-name "Partner Application" \
    --scope "/apis/mcp-api" \
    --state active
```

---

### 5.4.2 Rate Limiting 정책

Subscription Key 기반으로 Rate Limiting과 Quota를 적용할 수 있습니다.

**Rate Limiting 정책 예시:**
```xml
<policies>
    <inbound>
        <base />
        <!-- Rate Limiting: Subscription Key 기준 (분당 100회) -->
        <rate-limit-by-key 
            calls="100" 
            renewal-period="60" 
            counter-key="@(context.Subscription?.Key ?? context.Request.IpAddress)"
            increment-condition="@(context.Response.StatusCode &gt;= 200 &amp;&amp; context.Response.StatusCode &lt; 300)" />
        
        <!-- Quota: 일일 호출 제한 (10,000회/일) -->
        <quota-by-key 
            calls="10000" 
            renewal-period="86400" 
            counter-key="@(context.Subscription?.Key ?? context.Request.IpAddress)" />
        
        <!-- 요청 로깅 (Subscription Key 기반) -->
        <set-header name="X-Request-Source" exists-action="override">
            <value>@(context.Subscription?.Name ?? "anonymous")</value>
        </set-header>
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <!-- Rate Limit 정보 헤더 추가 -->
        <set-header name="X-RateLimit-Limit" exists-action="override">
            <value>100</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>
```

**주요 정책 설명:**
| 정책 | 설명 |
|-----|------|
| `rate-limit-by-key` | 지정된 키(Subscription Key) 기준으로 분당 호출 수 제한 |
| `quota-by-key` | 일/월 단위 총 호출 수 제한 |
| `counter-key` | Rate Limit 카운터 기준 키 (Subscription Key 또는 IP) |

---

### 5.4.3 API Key 테스트

Subscription Key를 사용하여 API를 호출합니다.

**cURL 예시:**
```bash
# Health Check
curl -X GET "https://<apim-name>.azure-api.net/mcp/health" \
     -H "Ocp-Apim-Subscription-Key: <your-subscription-key>"

# Tools List
curl -X GET "https://<apim-name>.azure-api.net/mcp/tools" \
     -H "Ocp-Apim-Subscription-Key: <your-subscription-key>" \
     -H "Content-Type: application/json"

# Tool 실행
curl -X POST "https://<apim-name>.azure-api.net/mcp/tools" \
     -H "Ocp-Apim-Subscription-Key: <your-subscription-key>" \
     -H "Content-Type: application/json" \
     -d '{"name": "echo", "arguments": {"message": "Hello from API Key!"}}'
```

**Python 예시:**
```python
import requests

apim_url = "https://<apim-name>.azure-api.net"
subscription_key = "<your-subscription-key>"

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

# Health Check
response = requests.get(f"{apim_url}/mcp/health", headers=headers)
print(response.json())

# Rate Limit 헤더 확인
print(f"X-RateLimit-Limit: {response.headers.get('X-RateLimit-Limit')}")
```

---

### 5.4.4 API Key vs JWT 비교

| 항목 | API 키 (Subscription Key) | JWT (OAuth 2.0) |
|------|---------------------------|-----------------|
| **설정 복잡도** | 🟢 매우 낮음 | 🟡 중간 |
| **보안 수준** | 🟡 중간 (IP 제한 필요) | 🟢 높음 |
| **사용자 식별** | ❌ 불가 (키 기반) | ✅ 가능 (클레임 기반) |
| **토큰 만료** | ❌ 수동 갱신 | ✅ 자동 만료 |
| **세분화된 권한** | ❌ 제한적 | ✅ 스코프/역할 기반 |
| **감사 추적** | 🟡 키 단위 | 🟢 사용자 단위 |
| **Rate Limiting** | ✅ 내장 지원 | 🟡 별도 구성 필요 |
| **사용 사례** | 내부 서비스, B2B 파트너 | 사용자 인증, 외부 앱 |

---

### 5.4.5 JWT + API Key 조합 사용

실무에서는 두 가지를 조합하여 사용하는 것이 일반적입니다:

```
┌─────────────────────────────────────────────────────────────┐
│                    API 요청 헤더                             │
├─────────────────────────────────────────────────────────────┤
│  Authorization: Bearer <JWT_TOKEN>     ← 사용자/서비스 인증  │
│  Ocp-Apim-Subscription-Key: <API_KEY>  ← Rate Limiting/추적 │
└─────────────────────────────────────────────────────────────┘
```

**역할 분담:**
- **JWT Token**: 인증(Authentication) - 누가 호출하는가?
- **Subscription Key**: 제어(Control) - 얼마나 호출할 수 있는가?

이 조합으로 보안과 사용량 관리를 모두 달성할 수 있습니다.

<a id='6'></a>
# 6️⃣ 모니터링 및 최적화

Application Insights와 APIM Diagnostics를 활용하여 MCP 서버를 모니터링하고 최적화합니다.

## 학습 목표
- Application Insights 쿼리
- APIM 사용량 분석
- 성능 최적화

## 6.1 Application Insights 쿼리

Kusto 쿼리를 사용하여 로그를 분석합니다.

In [24]:
# Application Insights 쿼리 예제
print("📊 유용한 Application Insights 쿼리:")
print()
print("=" * 80)
print("1️⃣ MCP Tool 호출 통계")
print("=" * 80)
print("""
requests
| where timestamp > ago(24h)
| where url contains "/mcp/tools"
| where success == true
| summarize Count = count() by name
| order by Count desc
""")
print()
print("=" * 80)
print("2️⃣ 응답 시간 분석")
print("=" * 80)
print("""
requests
| where timestamp > ago(24h)
| where url contains "/mcp/"
| summarize 
    avg(duration),
    percentile(duration, 50),
    percentile(duration, 95),
    percentile(duration, 99)
    by operation_Name
""")
print()
print("=" * 80)
print("3️⃣ 에러율 분석")
print("=" * 80)
print("""
requests
| where timestamp > ago(24h)
| where url contains "/mcp/"
| summarize 
    Total = count(),
    Errors = countif(success == false),
    ErrorRate = (countif(success == false) * 100.0) / count()
    by bin(timestamp, 1h)
| render timechart
""")
print()
print("=" * 80)
print("4️⃣ 사용자별 사용량")
print("=" * 80)
print("""
requests
| where timestamp > ago(7d)
| where url contains "/mcp/"
| extend userId = tostring(customDimensions["X-User-Id"])
| where isnotempty(userId) and userId != "anonymous"
| summarize 
    CallCount = count(),
    UniqueTools = dcount(operation_Name)
    by userId
| order by CallCount desc
""")
print()
print("=" * 80)
print("5️⃣ Rate Limit 적중 분석")
print("=" * 80)
print("""
traces
| where timestamp > ago(24h)
| where message contains "rate limit"
| summarize Count = count() by bin(timestamp, 1h), client_IP
| render timechart
""")
print()
print("🔗 Application Insights 접근:")
print("   Azure Portal > Application Insights 리소스 > Logs")

📊 유용한 Application Insights 쿼리:

1️⃣ MCP Tool 호출 통계

requests
| where timestamp > ago(24h)
| where url contains "/mcp/tools"
| where success == true
| summarize Count = count() by name
| order by Count desc


2️⃣ 응답 시간 분석

requests
| where timestamp > ago(24h)
| where url contains "/mcp/"
| summarize 
    avg(duration),
    percentile(duration, 50),
    percentile(duration, 95),
    percentile(duration, 99)
    by operation_Name


3️⃣ 에러율 분석

requests
| where timestamp > ago(24h)
| where url contains "/mcp/"
| summarize 
    Total = count(),
    Errors = countif(success == false),
    ErrorRate = (countif(success == false) * 100.0) / count()
    by bin(timestamp, 1h)
| render timechart


4️⃣ 사용자별 사용량

requests
| where timestamp > ago(7d)
| where url contains "/mcp/"
| extend userId = tostring(customDimensions["X-User-Id"])
| where isnotempty(userId) and userId != "anonymous"
| summarize 
    CallCount = count(),
    UniqueTools = dcount(operation_Name)
    by userId
| order by CallCount

---
<a id='6.2'></a>
## 6.2 APIM 진단 및 성능 모니터링

APIM의 Analytics와 진단 설정을 확인합니다.

In [25]:
# APIM 진단 설정 확인
print("🔍 APIM 진단 설정 확인:")
print()

# 1. APIM Analytics 확인
print("1️⃣ APIM Analytics 데이터:")
print("""
Azure Portal > API Management 서비스 > Analytics 탭에서:
  - API 호출 통계
  - 응답 시간 분포
  - 지리적 분포
  - 가장 많이 호출된 API
  - 에러율 추이
""")
print()

# 2. 진단 로그 설정
print("2️⃣ 진단 로그 설정:")
print("""
Azure Portal > API Management > Diagnostic settings > Add diagnostic setting

체크할 항목:
  ✅ GatewayLogs - 모든 API 요청/응답 로그
  ✅ WebSocketConnectionLogs - WebSocket 연결 로그
  ✅ AllMetrics - 성능 메트릭

전송 대상:
  ✅ Send to Log Analytics workspace
  ✅ Archive to storage account (선택사항)
""")
print()

# 3. APIM 로그 쿼리
print("3️⃣ APIM Gateway Logs 쿼리 (Log Analytics):")
print("""
ApiManagementGatewayLogs
| where TimeGenerated > ago(24h)
| where ApiId contains "mcp"
| summarize 
    TotalCalls = count(),
    AvgDuration = avg(TotalTime),
    ErrorCount = countif(ResponseCode >= 400)
    by bin(TimeGenerated, 1h), OperationId
| render timechart
""")
print()

# 4. 성능 메트릭 쿼리
print("4️⃣ 성능 메트릭 쿼리:")
print("""
AzureMetrics
| where ResourceProvider == "MICROSOFT.APIMANAGEMENT"
| where TimeGenerated > ago(24h)
| where MetricName in ("Requests", "Duration", "Capacity")
| summarize avg(Average) by MetricName, bin(TimeGenerated, 1h)
| render timechart
""")
print()

print("✅ APIM 모니터링 설정 완료!")
print("💡 Tip: Azure Monitor Alerts를 설정하여 임계값 초과 시 알림을 받을 수 있습니다.")

🔍 APIM 진단 설정 확인:

1️⃣ APIM Analytics 데이터:

Azure Portal > API Management 서비스 > Analytics 탭에서:
  - API 호출 통계
  - 응답 시간 분포
  - 지리적 분포
  - 가장 많이 호출된 API
  - 에러율 추이


2️⃣ 진단 로그 설정:

Azure Portal > API Management > Diagnostic settings > Add diagnostic setting

체크할 항목:
  ✅ GatewayLogs - 모든 API 요청/응답 로그
  ✅ WebSocketConnectionLogs - WebSocket 연결 로그
  ✅ AllMetrics - 성능 메트릭

전송 대상:
  ✅ Send to Log Analytics workspace
  ✅ Archive to storage account (선택사항)


3️⃣ APIM Gateway Logs 쿼리 (Log Analytics):

ApiManagementGatewayLogs
| where TimeGenerated > ago(24h)
| where ApiId contains "mcp"
| summarize 
    TotalCalls = count(),
    AvgDuration = avg(TotalTime),
    ErrorCount = countif(ResponseCode >= 400)
    by bin(TimeGenerated, 1h), OperationId
| render timechart


4️⃣ 성능 메트릭 쿼리:

AzureMetrics
| where ResourceProvider == "MICROSOFT.APIMANAGEMENT"
| where TimeGenerated > ago(24h)
| where MetricName in ("Requests", "Duration", "Capacity")
| summarize avg(Average) by MetricName, bin(TimeGenerated, 1

---
<a id='6.3'></a>
## 6.3 성능 최적화 팁

MCP + APIM 아키텍처의 성능을 최적화하는 방법입니다.

In [26]:
# 성능 최적화 가이드
print("⚡ MCP + APIM 성능 최적화 체크리스트:")
print()

print("=" * 80)
print("1️⃣ Azure Functions 최적화")
print("=" * 80)
print("""
✅ 적절한 Hosting Plan 선택:
   - Consumption Plan: 간헐적 사용, 낮은 비용 (현재 설정)
   - Premium Plan: 예측 가능한 성능, VNet 통합
   - Dedicated Plan: 지속적인 워크로드

✅ 함수 최적화:
   - 불필요한 모듈 import 제거
   - 전역 변수로 재사용 가능한 객체 캐싱
   - 비동기 처리 활용 (async/await)
   - 메모리 사용량 모니터링

✅ Cold Start 최소화:
   - Premium Plan 사용 (Always Ready instances)
   - 함수 워밍업 스케줄 설정
   - 의존성 최소화
""")
print()

print("=" * 80)
print("2️⃣ APIM 최적화")
print("=" * 80)
print("""
✅ Caching 정책 활용:
   <cache-lookup vary-by-developer="false" vary-by-developer-groups="false" />
   <cache-store duration="3600" />

✅ Backend 연결 최적화:
   - Connection pooling 활성화
   - Timeout 적절히 설정
   - Retry 정책 구성

✅ Rate Limiting 세밀화:
   - 사용자별/구독별 제한 설정
   - Burst 허용량 조정
   - Quota 정책 활용

✅ 응답 압축:
   - gzip 압축 활성화
   - 큰 페이로드 최적화
""")
print()

print("=" * 80)
print("3️⃣ MCP 프로토콜 최적화")
print("=" * 80)
print("""
✅ Tool 설계:
   - Tool 실행 시간 최소화
   - 무거운 작업은 비동기 처리
   - 결과 크기 제한 (페이징 고려)

✅ 에러 핸들링:
   - 명확한 에러 메시지 제공
   - Retry 가능한 에러 구분
   - Circuit breaker 패턴 고려

✅ 보안과 성능 균형:
   - JWT 캐싱 (토큰 재사용)
   - 불필요한 검증 최소화
   - JWKS 캐싱 활용
""")
print()

print("=" * 80)
print("4️⃣ 모니터링 및 알림")
print("=" * 80)
print("""
✅ 주요 메트릭 추적:
   - 응답 시간 (p50, p95, p99)
   - 에러율 (4xx, 5xx)
   - 처리량 (requests/sec)
   - 리소스 사용률 (CPU, Memory)

✅ Alert 설정:
   - 응답 시간 > 1초
   - 에러율 > 5%
   - Rate limit 적중 빈도
   - 함수 실행 실패

✅ 대시보드 구성:
   - Azure Dashboard에 주요 메트릭 시각화
   - Grafana/Power BI 연동 고려
""")
print()

print("✅ 성능 최적화 가이드 완료!")
print("💡 Tip: 주기적으로 성능 테스트를 실행하고 병목 구간을 식별하세요.")

⚡ MCP + APIM 성능 최적화 체크리스트:

1️⃣ Azure Functions 최적화

✅ 적절한 Hosting Plan 선택:
   - Consumption Plan: 간헐적 사용, 낮은 비용 (현재 설정)
   - Premium Plan: 예측 가능한 성능, VNet 통합
   - Dedicated Plan: 지속적인 워크로드

✅ 함수 최적화:
   - 불필요한 모듈 import 제거
   - 전역 변수로 재사용 가능한 객체 캐싱
   - 비동기 처리 활용 (async/await)
   - 메모리 사용량 모니터링

✅ Cold Start 최소화:
   - Premium Plan 사용 (Always Ready instances)
   - 함수 워밍업 스케줄 설정
   - 의존성 최소화


2️⃣ APIM 최적화

✅ Caching 정책 활용:
   <cache-lookup vary-by-developer="false" vary-by-developer-groups="false" />
   <cache-store duration="3600" />

✅ Backend 연결 최적화:
   - Connection pooling 활성화
   - Timeout 적절히 설정
   - Retry 정책 구성

✅ Rate Limiting 세밀화:
   - 사용자별/구독별 제한 설정
   - Burst 허용량 조정
   - Quota 정책 활용

✅ 응답 압축:
   - gzip 압축 활성화
   - 큰 페이로드 최적화


3️⃣ MCP 프로토콜 최적화

✅ Tool 설계:
   - Tool 실행 시간 최소화
   - 무거운 작업은 비동기 처리
   - 결과 크기 제한 (페이징 고려)

✅ 에러 핸들링:
   - 명확한 에러 메시지 제공
   - Retry 가능한 에러 구분
   - Circuit breaker 패턴 고려

✅ 보안과 성능 균형:
   - JWT 캐싱 (토큰 재사용)
   - 불필요한 검증 최소화
   - JWKS 캐싱 활용


4️⃣ 모니터링 및 알림

---
<a id='6.4'></a>
## 6.4 환경 변수 내보내기 (mcp-llm-test.ipynb 연동용)

`mcp-llm-test.ipynb` 노트북에서 이 워크샵의 설정값을 사용하려면 아래 셀을 실행하여 `.env` 파일 형식으로 환경 변수를 출력하세요.

In [27]:
# mcp-llm-test.ipynb에서 사용할 설정을 JSON 파일로 저장
CONFIG_FILE = "workshop_config.json"

print("📋 mcp-llm-test.ipynb 연동을 위한 설정 저장")
print("=" * 80)

# JSON 파일로 저장
with open(CONFIG_FILE, 'w', encoding='utf-8') as f:
    json.dump(WORKSHOP_CONFIG, f, indent=2, ensure_ascii=False)

print(f"✅ 설정 파일 저장 완료: {CONFIG_FILE}")
print()

# 저장된 주요 값 표시
print("📌 저장된 주요 설정값:")
print("-" * 60)
print(f"   🌐 apim_gateway_url: {WORKSHOP_CONFIG.get('apim_gateway_url', '(없음)')}")
print(f"   🔑 apim_subscription_key: {WORKSHOP_CONFIG.get('apim_subscription_key', '(없음)')[:10] + '...' if WORKSHOP_CONFIG.get('apim_subscription_key') else '(없음)'}")
print(f"   🏢 tenant_id: {WORKSHOP_CONFIG.get('tenant_id', '(없음)')[:8] + '...' if WORKSHOP_CONFIG.get('tenant_id') else '(없음)'}")
print(f"   🎯 entra_api_app_id: {WORKSHOP_CONFIG.get('entra_api_app_id', '(없음)')[:8] + '...' if WORKSHOP_CONFIG.get('entra_api_app_id') else '(없음)'}")
print(f"   📱 entra_client_app_id: {WORKSHOP_CONFIG.get('entra_client_app_id', '(없음)')[:8] + '...' if WORKSHOP_CONFIG.get('entra_client_app_id') else '(없음)'}")
print(f"   🔐 entra_client_secret: {WORKSHOP_CONFIG.get('entra_client_secret', '(없음)')[:4] + '***...' if WORKSHOP_CONFIG.get('entra_client_secret') else '(없음)'}")
print("-" * 60)
print()
print("💡 mcp-llm-test.ipynb에서 자동으로 이 파일을 읽어옵니다.")
print("=" * 80)

📋 mcp-llm-test.ipynb 연동을 위한 설정 저장
✅ 설정 파일 저장 완료: workshop_config.json

📌 저장된 주요 설정값:
------------------------------------------------------------
   🌐 apim_gateway_url: https://mcp-dev-apim-i6ht.azure-api.net
   🔑 apim_subscription_key: 90c98e4841...
   🏢 tenant_id: 38bd4dc5...
   🎯 entra_api_app_id: 56a56150...
   📱 entra_client_app_id: 3c1271cc...
   🔐 entra_client_secret: gNl8***...
------------------------------------------------------------

💡 mcp-llm-test.ipynb에서 자동으로 이 파일을 읽어옵니다.


---
<a id='7'></a>
# 7️⃣ 리소스 정리 (Cleanup)

실습이 끝나면 생성된 Azure 리소스를 정리하여 불필요한 비용을 방지합니다.

## 📁 별도 노트북 사용

리소스 정리는 **[cleanup.ipynb](cleanup.ipynb)** 노트북에서 수행합니다.

### 포함 기능:
- 🗑️ **리소스 그룹 삭제** - 전체 리소스 한 번에 삭제
- 🔧 **개별 리소스 삭제** - APIM, Functions 등 선택적 삭제
- 🔐 **Entra ID 앱 삭제** - MCP-API, MCP-Client 앱 정리
- 💰 **비용 관리 팁** - 리소스 유지 시 비용 절감 방법

### 사용 방법:
1. `cleanup.ipynb` 파일 열기
2. 삭제할 리소스 선택
3. 해당 셀 실행

⚠️ **주의**: 리소스 삭제는 되돌릴 수 없습니다. 중요한 데이터는 백업하세요.

---

# 🎉 워크샵 완료!

축하합니다! Azure API Management와 Azure Functions를 활용한 MCP 서버 구축 워크샵을 완료하셨습니다.

## 📚 학습한 내용

이 워크샵에서 다음 내용을 학습하고 실습했습니다:

### 1️⃣ Azure Functions MCP 서버 구축
- Model Context Protocol (MCP) 2024-11-05 표준 구현
- Python 3.12 기반 HTTP Trigger 함수 개발
- 3가지 MCP Tool 구현 (echo, get_current_time, calculate)
- Health check 및 정보 조회 엔드포인트

### 2️⃣ Azure API Management 게이트웨이 설정
- APIM Basicv2 SKU를 사용한 엔터프라이즈 게이트웨이 구성
- CORS, Rate Limiting, Backend 연결 정책 설정
- API 정의 및 운영 관리

### 3️⃣ REST API ↔ MCP 변환
- 클라이언트 REST 요청을 MCP JSON-RPC로 변환
- MCP 응답을 RESTful 형식으로 변환
- 헤더 전달 및 컨텍스트 유지

### 4️⃣ OpenID Connect 인증 통합
- Entra ID (Azure AD) App Registration 구성
- OAuth 2.0 Client Credentials Flow 구현
- JWT 토큰 검증 정책 (validate-jwt)
- Bearer 토큰 기반 보안 적용

### 5️⃣ 외부 Entra ID 연동
- B2B Guest User 초대 및 관리
- 멀티 테넌트 인증 시나리오
- 외부 조직과의 안전한 API 공유

### 6️⃣ 모니터링 및 최적화
- Application Insights 쿼리 (Kusto)
- APIM Analytics 및 진단 로그
- 성능 최적화 체크리스트
- 비용 관리 전략

### 7️⃣ 인프라스트럭처 as Code
- Bicep 템플릿을 사용한 자동화된 배포
- Azure CLI를 활용한 리소스 관리
- 환경 변수 기반 구성

---

## 🚀 다음 단계

워크샵을 완료한 후 다음 활동을 고려해보세요:

### 🔧 프로젝트 확장
- **추가 MCP Tools 개발**: 실제 비즈니스 로직 구현 (데이터베이스 조회, 외부 API 호출 등)
- **WebSocket 지원**: 실시간 스트리밍을 위한 MCP over WebSocket 구현
- **AI 에이전트 통합**: OpenAI Assistant API나 Azure OpenAI와 MCP 연동

### 🔐 보안 강화
- **Managed Identity**: APIM ↔ Functions 간 Managed Identity 인증 적용
- **API Key Rotation**: 자동 키 로테이션 메커니즘 구현
- **IP Filtering**: APIM에서 허용된 IP만 접근 가능하도록 설정

### 📊 운영 개선
- **CI/CD 파이프라인**: GitHub Actions/Azure DevOps를 사용한 자동 배포
- **Multi-Region 배포**: 글로벌 사용자를 위한 다중 지역 배포
- **Auto-scaling**: Premium Functions Plan으로 업그레이드하여 자동 확장

### 🧪 고급 시나리오
- **GraphQL Gateway**: APIM에서 GraphQL 엔드포인트 노출
- **Event-driven Architecture**: Event Grid/Service Bus와 통합
- **Microservices Mesh**: 여러 MCP 서버를 오케스트레이션

---

## 📖 참고 자료

### MCP (Model Context Protocol)
- [MCP 공식 문서](https://modelcontextprotocol.io/)
- [MCP Specification](https://spec.modelcontextprotocol.io/)
- [MCP GitHub Repository](https://github.com/modelcontextprotocol)

### Azure API Management
- [APIM 공식 문서](https://learn.microsoft.com/azure/api-management/)
- [APIM 정책 참조](https://learn.microsoft.com/azure/api-management/api-management-policies)
- [APIM 샘플 정책](https://github.com/Azure/api-management-policy-snippets)

### Azure Functions
- [Functions 공식 문서](https://learn.microsoft.com/azure/azure-functions/)
- [Python Functions 개발자 가이드](https://learn.microsoft.com/azure/azure-functions/functions-reference-python)
- [Functions Best Practices](https://learn.microsoft.com/azure/azure-functions/functions-best-practices)

### Entra ID (Azure AD)
- [Entra ID 문서](https://learn.microsoft.com/entra/identity/)
- [OAuth 2.0 및 OpenID Connect](https://learn.microsoft.com/entra/identity-platform/v2-protocols)
- [B2B Collaboration](https://learn.microsoft.com/entra/external-id/what-is-b2b)

---

## 💬 피드백 및 기여

이 워크샵에 대한 피드백이나 개선 제안이 있으시면 언제든지 공유해주세요!

- GitHub Issues를 통한 버그 리포트
- Pull Request를 통한 코드 개선
- 커뮤니티 포럼에서 질문 및 토론

---

## 🙏 감사합니다!

이 워크샵을 완료해주셔서 감사합니다. Azure와 MCP를 활용한 엔터프라이즈 통합에 대해 유용한 지식을 얻으셨기를 바랍니다.

행운을 빕니다! 🚀

---

**마지막 업데이트**: 2024
**워크샵 버전**: 1.0
**제작**: Azure API Management + MCP Workshop